COS80024

DATA SCIENCE PROJECT 1

PROJECT 4: MOVIE RECOMMENDATION SYSTEM

# Task S3.7 Recommending Top-10 Movies

This task aims to apply each of the recommendation systems created to predict the ratings of all the unrated movies for each user and then recommend the movies of the highest 10 predicated ratings, as top‐10 movies, to each corresponding user.

Code Contributed by - Asif Rana, Student ID - 102866893

In [1]:
# Load the Python libraries
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import pickle
import time
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error
from sklearn import metrics
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.feature_extraction.text import TfidfVectorizer
import sklearn.metrics as sm
from sklearn.metrics import mean_absolute_error
import time

# Load Surprise libraries
from surprise import SVD
from surprise import Reader
from surprise import Dataset
from surprise import accuracy

## Using Content-based Recommendation 

In [2]:
# Importing the csv data files 
movies_metadata = pd.read_csv('metadata_integrated.csv',error_bad_lines=False, warn_bad_lines=True)
train_df = pd.read_csv('train_df.csv',error_bad_lines=False, warn_bad_lines=True)
movie_title = movies_metadata[['movieId','title']]

In [3]:
movie_title.head()

,movieId,title
0,1,Toy Story
1,2,Jumanji
2,3,Grumpier Old Men
3,4,Waiting to Exhale
4,5,Father of the Bride Part II


In [4]:
global_mean_rating = np.average(train_df.rating.values)

In [5]:
global_mean_rating

3.5385446087389933

# Change the filename below to your required csv file

In [6]:
#Load required csv file
test = pd.read_csv('unrated_movies.csv',error_bad_lines=False, warn_bad_lines=True)
test_df = test 
test_df1 = test 

## Conducting the preprocessing required

In [7]:
content_recom_df = movies_metadata[['movieId', 'overview','genres','title', 'production_companies', 'collectionName']]
content_recom_df['genres'] = content_recom_df['genres'].str.replace(',',' ').str.lower()
content_recom_df['concat_title'] = content_recom_df['title'].str.lower().str.replace(' ','') + content_recom_df['movieId'].astype(str)
content_recom_df['overview'] = content_recom_df['overview'].str.lower()
content_recom_df['collectionName'] = content_recom_df['collectionName'].str.replace(' ','').str.lower()
content_recom_df['production_companies'] = content_recom_df['production_companies'].str.replace(' ','').str.lower().str.replace(',',' ')

content_recom_df['corpus'] =  content_recom_df['genres'] + ' ' + content_recom_df['overview']  
content_recom_df['corpus'] = content_recom_df['corpus'].str.replace('[^\w\s]','') 
content_recom_df['corpus'] = content_recom_df['corpus'].str.replace('unknown','')

C:\Users\user\anaconda3New\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\user\anaconda3New\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\user\anaconda3New\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

In [8]:
#Define a TF-IDF Vectorizer Object. Removing all english stopwords
tfidf = TfidfVectorizer(stop_words='english')

#Constructing the required TF-IDF matrix by applying the fit_transform method on the overview feature
tfidf_matrix = tfidf.fit_transform(content_recom_df['corpus'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape

# Import linear_kernel to compute the dot product
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim_tfidf = linear_kernel(tfidf_matrix, tfidf_matrix)

In [9]:
# Reset index of your df and construct reverse mapping again
content_recom_df = content_recom_df.reset_index()
indices = pd.Series(content_recom_df.index, index=content_recom_df['movieId'])

In [10]:
train_df = train_df[['userId','movieId','rating']]
test_df1 = test_df1[['userId','movieId',]]

In [11]:
test_df1.head()

,userId,movieId
0,1,1
1,1,2
2,1,3
3,1,4
4,1,5


In [12]:
#5967523 entries
start = time.time()
users = test_df1.userId.unique()
k = 36 # Setting the value for the most similar movies number
start = time.time()
for i in range(len(users)):
    start1 = time.time()
    rated_movies = train_df.loc[(train_df['userId'] == users[i])].movieId.values
    unrated_movies = test_df1.loc[(test_df1['userId'] == users[i])].movieId.values
    for movie in rated_movies:
        idx = indices[movie]
        sim_scores = list(enumerate(cosine_sim_tfidf[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:k]
        movie_indices = [i[0] for i in sim_scores]
        movieIds = movies_metadata['movieId'].iloc[movie_indices].values    
        unrated_movies = list(set(movieIds).intersection(unrated_movies))
        if len(unrated_movies) > 0:
            for movie in unrated_movies:
                idx = indices[movie]
                sim_scores = list(enumerate(cosine_sim_tfidf[idx]))
                sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
                sim_scores = sim_scores[1:k]
                movieIds = movies_metadata['movieId'].iloc[movie_indices].values  
                movie_indices = [i[0] for i in sim_scores]
                
                movieIds = movies_metadata['movieId'].iloc[movie_indices].values  
                unrated_movies = list(set(movieIds).intersection(rated_movies))
                if len(unrated_movies)> 0:
                    weight_movies = [i[1] for i in sim_scores]
                    temp = pd.DataFrame()
                    temp2 = pd.DataFrame()
                    merged_rating_df = pd.DataFrame()
                    temp['movieId'] = movies_metadata['movieId'].iloc[movie_indices]
                    temp['weights'] = weight_movies
                    temp2['movieId'] = unrated_movies 
                    merged_rating_df = pd.merge(temp, temp2, how='inner', left_on=['movieId'], right_on = ['movieId'])
                    merged_rating_df = pd.merge(train_df.loc[(train_df['userId'] == users[i])], merged_rating_df , how='inner', left_on=['movieId'], right_on = ['movieId'])
                    merged_rating_df["weights*rating"] = merged_rating_df["weights"] * merged_rating_df["rating"]
                    weights = merged_rating_df['weights'].values
                    weights_rating = merged_rating_df['weights*rating'].values
                    prediction = sum(weights_rating)/(sum(weights))
                    test_df1.loc[(test_df1['userId'] == users[i]) & (test_df1['movieId'] == movie),'prediction'] = prediction     
    end1 = time.time()
    print('User : ', users[i])
    print(end1 - start1)  
end = time.time()
print(end - start)       

User :  1
6.179577350616455
User :  7
6.376026153564453
User :  31
4.580092668533325
User :  32
6.84277081489563
User :  36
7.323735475540161
User :  39
5.774334669113159
User :  73
31.653434991836548
User :  88
10.933228969573975
User :  96
4.75857400894165
User :  110
8.156384944915771
User :  111
11.636534929275513
User :  150
12.884124755859375
User :  161
7.202664136886597
User :  165
14.27046823501587
User :  186
6.802936792373657
User :  242
14.637351989746094
User :  254
7.096067190170288
User :  288
8.000920295715332
User :  310
4.968719005584717
User :  311
31.834481954574585
User :  325
6.099377632141113
User :  338
6.8968095779418945
User :  341
3.784182071685791
User :  386
8.722046136856079
User :  394
9.690805196762085
User :  452
25.955858945846558
User :  457
16.771798372268677
User :  461
16.271299600601196
User :  468
26.641526699066162
User :  485
3.2205779552459717
User :  487
8.52963924407959
User :  496
7.906928777694702
User :  511
3.534073829650879
User :  516


User :  412
8.5421302318573
User :  480
11.42505955696106
User :  557
5.758031129837036
User :  595
7.204547643661499
User :  611
6.106539964675903
User :  656
6.969477891921997
User :  75
6.573841571807861
User :  157
10.610129117965698
User :  167
5.826426029205322
User :  199
12.830968618392944
User :  355
9.66959285736084
User :  395
5.489603281021118
User :  482
6.754504680633545
User :  544
10.692658185958862
User :  599
7.8669044971466064
User :  626
6.485872507095337
User :  68
7.119714260101318
User :  282
6.682802438735962
User :  76
4.825940370559692
User :  83
7.57984471321106
User :  125
7.991799592971802
User :  134
9.582237243652344
User :  178
6.555346727371216
User :  196
6.0384604930877686
User :  222
8.024041175842285
User :  315
6.812717437744141
User :  352
6.115419864654541
User :  398
5.629469633102417
User :  424
4.2550132274627686
User :  519
6.537014961242676
User :  553
7.477149724960327
User :  615
11.500563859939575
User :  621
7.43295693397522
User :  671


User :  348
5.278719425201416
User :  393
7.4475603103637695
User :  644
2.7721023559570312
User :  12
6.900136470794678
User :  189
7.791696548461914
User :  215
5.0848565101623535
User :  368
5.221795082092285
User :  645
6.654276609420776
User :  515
6.865914821624756
User :  107
2.9539711475372314
User :  112
3.706390619277954
User :  181
4.171096563339233
User :  208
3.813424825668335
User :  274
4.978976726531982
User :  278
4.895063161849976
User :  334
4.098360776901245
User :  445
3.10463809967041
User :  506
4.242506742477417
User :  524
6.980649948120117
User :  583
3.5910422801971436
User :  609
6.478759288787842
User :  668
3.167357921600342
User :  210
7.35936164855957
User :  158
5.8558948040008545
User :  127
6.272205829620361
User :  160
7.34305739402771
User :  131
4.760310649871826
User :  140
6.550835371017456
User :  166
5.97555136680603
User :  234
6.44121527671814
User :  238
5.56695032119751
User :  255
6.863604784011841
User :  259
5.350890159606934
User :  260

In [13]:
recom_content_pred = test_df1

In [14]:
recom_content_pred = recom_content_pred.fillna(global_mean_rating)

In [15]:
recom_content_pred.prediction.unique()

array([3.53854461, 2.5       , 2.23397245, ..., 2.17840419, 1.90398833,
       3.48196351])

In [16]:
recom_content_pred = pd.merge(recom_content_pred, movie_title, how='left', left_on=['movieId'], right_on = ['movieId'])

In [17]:
sorted_pred = recom_content_pred.sort_values(by =['userId', 'prediction'], ascending= False)

In [18]:
sorted_pred.head()

,userId,movieId,prediction,title
2692778,671,490,5.0,Malice
2694094,671,2269,5.0,Indecent Proposal
2694492,671,2750,5.0,Radio Days
2694708,671,3021,5.0,The Funhouse
2694894,671,3263,5.0,White Men Can't Jump


## Printing the recommendations for Content-Based

In [19]:
for i in range(len(users)):
    top10movies = sorted_pred.loc[(sorted_pred['userId'] == users[i])]['title'].values
    top10movies = top10movies[:10]
    print('Recommendation for user: ', users[i])
    print('Movies: ',top10movies)

Recommendation for user:  1
Movies:  ['Stealth' 'Toy Story' 'Jumanji' 'Grumpier Old Men' 'Waiting to Exhale'
 'Father of the Bride Part II' 'Heat' 'Sabrina' 'Tom and Huck'
 'Sudden Death']
Recommendation for user:  7
Movies:  ['Jumanji' 'Grumpier Old Men' 'Waiting to Exhale'
 'Father of the Bride Part II' 'Heat' 'Sabrina' 'Tom and Huck'
 'Sudden Death' 'The American President' 'Dracula: Dead and Loving It']
Recommendation for user:  31
Movies:  ['My Super Ex-Girlfriend' 'Jason and the Argonauts' 'Jack & Sarah'
 'The Bridges of Madison County' 'Pollyanna'
 'Blood, Guts, Bullets and Octane' 'To Sir, with Love' 'Only the Strong'
 'Born Rich' 'Suicide Club']
Recommendation for user:  32
Movies:  ['Maverick' 'D3: The Mighty Ducks' 'Peter Pan' 'eXistenZ'
 'Any Given Sunday' 'Manhunter' 'Panic Room' 'Rollerball'
 'Spy Kids 3-D: Game Over' 'Avalon']
Recommendation for user:  36
Movies:  ['Star Trek V: The Final Frontier' 'Biloxi Blues' 'İtirazım Var'
 'Manhunter' 'Toy Story' 'Jumanji' 'Grumpie

Recommendation for user:  19
Movies:  ['Mask' "The Ballad of Ramblin' Jack" 'The Thin Red Line'
 'Goodbye, Columbus' 'Rudolph, the Red-Nosed Reindeer' 'A Midnight Clear'
 'Malice' 'For Your Consideration' 'Father of the Bride Part II'
 'Tom and Huck']
Recommendation for user:  35
Movies:  ['Toy Story' 'Jumanji' 'Grumpier Old Men' 'Waiting to Exhale'
 'Father of the Bride Part II' 'Heat' 'Sabrina' 'Tom and Huck'
 'Sudden Death' 'GoldenEye']
Recommendation for user:  61
Movies:  ['Zathura: A Space Adventure' 'The Brothers Grimm' 'Toy Story'
 'Grumpier Old Men' 'Waiting to Exhale' 'Father of the Bride Part II'
 'Heat' 'Sabrina' 'Tom and Huck' 'Sudden Death']
Recommendation for user:  99
Movies:  ['The Messenger' 'A Midnight Clear' 'Indecent Proposal' 'Malice'
 'Condorman' 'Radio Days' 'The Funhouse' "White Men Can't Jump"
 'Bound for Glory' 'Hollywood Ending']
Recommendation for user:  102
Movies:  ['Cinderella' 'Earthquake' 'American Pop' 'Night and Fog' 'Sunshine'
 'Into the Woods' 'Nap

Recommendation for user:  30
Movies:  ['A Midnight Clear' 'Room' 'Tai-Chi Master' 'Match Point'
 'For Your Consideration' 'Bound for Glory' 'Mask' 'Toy Story 3'
 'Hollywood Ending' 'Dirty Work']
Recommendation for user:  70
Movies:  ['Malice' 'Condorman' 'Indecent Proposal' 'Radio Days' 'The Funhouse'
 'Toy Story 2' "White Men Can't Jump" 'Bound for Glory' 'Hollywood Ending'
 "What's Up, Tiger Lily?"]
Recommendation for user:  90
Movies:  ['Malice' 'Rebel Without a Cause' 'Condorman' 'Indecent Proposal'
 'Radio Days' 'The Funhouse' 'Toy Story 2' "White Men Can't Jump"
 'Bound for Glory' 'Hollywood Ending']
Recommendation for user:  105
Movies:  ['Mulholland Falls' 'Death Wish 2' 'The Crush' 'Escape from L.A.'
 'The Cincinnati Kid' 'Biloxi Blues' 'Benny & Joon' 'Now You See Me'
 'Perfect Stranger' 'Toy Story']
Recommendation for user:  119
Movies:  ['Goodbye, Columbus' 'A Midnight Clear' 'The Thin Red Line'
 'Tai-Chi Master' 'Love! Valour! Compassion!' 'Mask'
 'For Your Consideration' '

Recommendation for user:  390
Movies:  ['Let It Ride' 'Malice' 'Rebel Without a Cause' 'Condorman'
 'Indecent Proposal' 'Stepmom' 'Radio Days' 'The Funhouse'
 'Hollywood Ending' "What's Up, Tiger Lily?"]
Recommendation for user:  423
Movies:  ['Donnie Brasco' 'Things Change' 'İtirazım Var'
 'Ghost Dog: The Way of the Samurai' 'Think Like a Man Too'
 'Bad Lieutenant' 'Tracers' 'Paul Blart: Mall Cop 2'
 'The Electric Horseman' 'We Own the Night']
Recommendation for user:  463
Movies:  ['Rudolph, the Red-Nosed Reindeer' 'The Thin Red Line' 'A Midnight Clear'
 'Let It Ride' "Max Keeble's Big Move" 'Mask' 'Goodbye, Columbus' 'Room'
 'Jumanji' 'Grumpier Old Men']
Recommendation for user:  522
Movies:  ['BASEketball' 'Stay Alive' 'Toy Story' 'Grumpier Old Men'
 'Waiting to Exhale' 'Father of the Bride Part II' 'Heat' 'Sabrina'
 'Tom and Huck' 'Sudden Death']
Recommendation for user:  580
Movies:  ['Smiles of a Summer Night' 'Malice' 'The Thin Red Line' 'The Funhouse'
 "What's Up, Tiger Lily?"

Recommendation for user:  587
Movies:  ['The World of Suzie Wong' 'Date and Switch'
 'Win a Date with Tad Hamilton!' 'Diamonds Are Forever' 'Mud'
 'Hostel: Part III' 'Breathless' 'Bugsy' 'Go' 'Casino']
Recommendation for user:  15
Movies:  ['Goodbye, Columbus' 'Smiles of a Summer Night' 'Grumpier Old Men'
 'Waiting to Exhale' 'Sabrina' 'Tom and Huck' 'Sudden Death'
 'Dracula: Dead and Loving It' 'Balto' 'Cutthroat Island']
Recommendation for user:  17
Movies:  ['Vincent & Theo' 'Star Trek V: The Final Frontier' 'City of Hope'
 'The Color of Money' 'Vincent' 'Falling Down' 'The Town' 'Twins'
 'The Reader' 'Beautiful Thing']
Recommendation for user:  22
Movies:  ['Æon Flux' 'I, Robot' 'Kafka' 'The Bucket List' 'Eye of the Beholder'
 "Big Momma's House" 'The Glimmer Man' 'This Island Earth'
 'Cruel Intentions' '28 Weeks Later']
Recommendation for user:  152
Movies:  ['Stepmom' 'Tai-Chi Master' 'A Midnight Clear' "Max Keeble's Big Move"
 'Radio Days' 'The Thin Red Line' 'Rebel Without a Ca

...

Recommendation for user:  139
Movies:  ['Tea with Mussolini' 'King Ralph' 'Never Let Me Go' 'Toy Story' 'Jumanji'
 'Grumpier Old Men' 'Waiting to Exhale' 'Father of the Bride Part II'
 'Heat' 'Sabrina']
Recommendation for user:  153
Movies:  ['Malice' 'Condorman' 'Indecent Proposal' 'Radio Days' 'The Funhouse'
 'Toy Story 2' "White Men Can't Jump" 'Bound for Glory'
 "The Ballad of Ramblin' Jack" 'Hollywood Ending']
Recommendation for user:  163
Movies:  ['History of the World: Part I' 'Jumanji' 'Grumpier Old Men'
 'Waiting to Exhale' 'Father of the Bride Part II' 'Heat' 'Sabrina'
 'Tom and Huck' 'Sudden Death' 'GoldenEye']
Recommendation for user:  194
Movies:  ['Ong-Bak: The Thai Warrior' 'The Paleface' 'Pathfinder' 'Toy Story'
 'Jumanji' 'Grumpier Old Men' 'Waiting to Exhale'
 'Father of the Bride Part II' 'Heat' 'Sabrina']
Recommendation for user:  200
Movies:  ['Let It Ride' 'Goodbye, Columbus' 'Mask' 'Smiles of a Summer Night'
 "What's Up, Tiger Lily?" 'Radio Days' 'Stepmom' 'Grum

Recommendation for user:  453
Movies:  ['The Mortal Instruments: City of Bones' 'Tomorrow Never Dies'
 'The Wolverine' 'Immortals' 'Robin Hood: Prince of Thieves'
 'City of Angels' 'The Gate' 'The Raven' 'The Punisher' 'Dragonslayer']
Recommendation for user:  184
Movies:  ['Malice' 'Rebel Without a Cause' 'Condorman' 'Indecent Proposal'
 'Radio Days' 'The Funhouse' 'Toy Story 2' "White Men Can't Jump"
 'Bound for Glory' 'Hollywood Ending']
Recommendation for user:  357
Movies:  ['Malice' 'Rebel Without a Cause' 'Condorman' 'Indecent Proposal'
 'Radio Days' 'The Funhouse' 'Toy Story 2' "White Men Can't Jump"
 'Bound for Glory' 'Hollywood Ending']
Recommendation for user:  53
Movies:  ["Jacob's Ladder" 'Heat' 'The War' 'Midnight Cowboy'
 'The Muppet Christmas Carol' 'Band of the Hand' 'Biloxi Blues'
 'Murmur of the Heart' 'My Best Friend' 'Taxi to the Dark Side']
Recommendation for user:  250
Movies:  ['Race to Witch Mountain' "We're Back! A Dinosaur's Story" 'Heat'
 'The War' 'The Fift

Recommendation for user:  158
Movies:  ["April Fool's Day" 'MouseHunt' 'Lock, Stock and Two Smoking Barrels' 'Go'
 'Frankenstein' 'The Sixth Sense' 'Dead Calm' "River's Edge"
 'Tango & Cash' 'Exodus']
Recommendation for user:  127
Movies:  ['Romance' 'Mrs. Winterbourne' 'Nine Months' 'Killer'
 'Father of the Bride' "It's a Wonderful Life" 'George of the Jungle'
 'Niagara' 'Miss Congeniality' 'Two Ninas']
Recommendation for user:  160
Movies:  ['Screwed' 'The Baby-Sitters Club' 'Mr. Deeds Goes to Town'
 'Ministry of Fear' 'Toy Story' 'Jumanji' 'Grumpier Old Men'
 'Waiting to Exhale' 'Father of the Bride Part II' 'Heat']
Recommendation for user:  131
Movies:  ['Being Human' 'Feeling Minnesota' 'Nowhere' 'Deadtime Stories'
 'The Masque of the Red Death' 'The Hours' 'Wake of the Red Witch'
 'Red River' 'Kwaidan' 'The Man with One Red Shoe']
Recommendation for user:  140
Movies:  ['Sense and Sensibility' 'Killer' 'Mrs. Winterbourne'
 'Father of the Bride' 'George of the Jungle' 'Romance'
 '

Recommendation for user:  625
Movies:  ['The 400 Blows' 'Shoot the Moon' 'Madame Sousatzka' 'Ragtime'
 'Away from Her' 'Toy Story' 'Jumanji' 'Grumpier Old Men'
 'Waiting to Exhale' 'Father of the Bride Part II']
Recommendation for user:  117
Movies:  ['The Pirate Movie' 'Hereafter' 'Bed of Roses' 'Before Midnight'
 'Tea with Mussolini' 'Crash' 'Toy Story' 'Jumanji' 'Grumpier Old Men'
 'Waiting to Exhale']
Recommendation for user:  169
Movies:  ["Max Keeble's Big Move" 'Goodbye, Columbus' 'The Thin Red Line' 'Room'
 'Radio Days' 'Malice' 'Rebel Without a Cause' 'Indecent Proposal'
 'The Funhouse' "White Men Can't Jump"]
Recommendation for user:  331
Movies:  ['Mighty Morphin Power Rangers: The Movie' 'Forrest Gump' 'Foxfire'
 'Turbo: A Power Rangers Movie' 'The Rapture' 'Phantasm' 'Bully'
 'The Serpent and the Rainbow' 'Equilibrium' 'The Butterfly Effect']
Recommendation for user:  347
Movies:  ['Toy Story' 'Jumanji' 'Grumpier Old Men' 'Waiting to Exhale'
 'Father of the Bride Part II' 

Recommendation for user:  65
Movies:  ['The Swan Princess' 'Princess Caraboo' 'The Princess Bride'
 'History of the World: Part I' 'The Mummy' 'The 7th Voyage of Sinbad'
 'Shanghai Noon' 'Sinbad and the Eye of the Tiger'
 "Bill & Ted's Bogus Journey"
 'Spacehunter: Adventures in the Forbidden Zone']
Recommendation for user:  638
Movies:  ['Sound of My Voice' 'The Crow: City of Angels' 'Help!' 'The One' 'Sahara'
 'The Man Who Fell to Earth' 'The Matrix Revolutions' 'Gunga Din'
 "Frank Herbert's Dune" 'The Ice Pirates']
Recommendation for user:  14
Movies:  ['Toy Story' 'Jumanji' 'Grumpier Old Men' 'Waiting to Exhale'
 'Father of the Bride Part II' 'Heat' 'Sabrina' 'Tom and Huck'
 'Sudden Death' 'GoldenEye']
Recommendation for user:  269
Movies:  ['Toy Story' 'Jumanji' 'Grumpier Old Men' 'Waiting to Exhale'
 'Father of the Bride Part II' 'Heat' 'Sabrina' 'Tom and Huck'
 'Sudden Death' 'GoldenEye']
Recommendation for user:  322
Movies:  ['The White Stripes: Under Great White Northern Ligh

## Collaborative Filters

### Creating datasets for collaborative filters

In [20]:
#Remove all information except User ID, Movie ID and Title
test_data = test[['userId','movieId','rating']]

In [21]:
# Read the data into a Surprise dataset
reader = Reader(rating_scale = (1, 5))
data_test = Dataset.load_from_df(test_data, reader)

# Build full trainset
data_test = data_test.build_full_trainset()

# Create the trainset and testset
data_testset = data_test.build_testset()

# Model-Based

In [22]:
#loading model pickle file
filename1 = 'model-based_CF.pickle'
model_cf = pickle.load(open(filename1, 'rb'))

In [23]:
# Predicting rating for the dataset
pred_rating = model_cf.test(data_testset)

In [24]:
# Saving predictions 
predictions = []
for i in range(len(pred_rating)):
    temp = pred_rating[i].est
    temp1 = round(temp,2)
    predictions.append(temp1)

In [25]:
# Converting to dataframe
predictions_df = pd.DataFrame(predictions, columns=['predictions'])

In [26]:
# Loading predictions onto dataset
model_cf_df = test_data.join(predictions_df)

In [27]:
#Dropping previous 'rating' column
model_cf_df  = model_cf_df.drop(columns=['rating'])

In [28]:
model_cf_df

,userId,movieId,predictions
0,1,1,2.99
1,1,2,2.70
2,1,3,3.06
3,1,4,1.06
4,1,5,2.14
...,...,...,...
5967518,227,161944,4.17
5967519,227,162542,4.59
5967520,227,162672,3.70
5967521,227,163056,4.05


### Printing the recommendations for Model-Based Collaborative Filtering

In [70]:
model_cf_df = pd.merge(model_cf_df, movie_title, how='left', left_on=['movieId'], right_on = ['movieId'])

In [71]:
sorted_pred = model_cf_df.sort_values(by =['userId', 'predictions'], ascending= False)

In [72]:
users = model_cf_df.userId.unique()
for i in range(len(users)):
    top10movies = sorted_pred.loc[(sorted_pred['userId'] == users[i])]['title'].values
    top10movies = top10movies[:10]
    print('Recommendation for user: ', users[i])
    print('MovieIds: ',top10movies)

Recommendation for user:  1
MovieIds:  ["Hachi: A Dog's Tale" 'Paris is Burning' 'Chungking Express' 'Smoke'
 'Man Facing Southeast' 'Stand by Me' 'All About Eve' 'The Matrix'
 'The Barbarian Invasions' 'The Breakfast Club']
Recommendation for user:  7
MovieIds:  ['The Godfather' 'The Lord of the Rings: The Fellowship of the Ring'
 'Fight Club' 'Citizen Kane' 'Cool Hand Luke' 'Memento'
 'My Left Foot: The Story of Christy Brown' 'The Godfather: Part II'
 'The Philadelphia Story' 'All About Eve']
Recommendation for user:  31
MovieIds:  ['Braveheart' 'Naked' 'Independence Day' 'Brazil' 'Jerry Maguire'
 'The French Connection' 'Lock, Stock and Two Smoking Barrels'
 'The Sixth Sense' 'O Brother, Where Art Thou?' 'Midnight in Paris']
Recommendation for user:  32
MovieIds:  ['Taxi Driver' 'The Godfather: Part II' 'Back to the Future'
 'Indiana Jones and the Last Crusade' 'Titanic' 'Eyes Wide Shut'
 'Ghostbusters' 'Shutter Island' 'The Grand Budapest Hotel'
 'The Twilight Saga: Breaking Dawn 

Recommendation for user:  596
MovieIds:  ['Crumb' 'Harold and Maude' "The Emperor's New Groove"
 "Hachi: A Dog's Tale" 'The Conversation' 'The Best Years of Our Lives'
 'Up' 'Inglourious Basterds' 'Amadeus' "Charlotte's Web"]
Recommendation for user:  601
MovieIds:  ['Taxi Driver' 'Fargo' 'Being John Malkovich' 'It Happened One Night'
 'Apocalypse Now' 'The Silence of the Lambs' 'Vertigo' "Schindler's List"
 'The Maltese Falcon' 'Psycho']
Recommendation for user:  607
MovieIds:  ['Heat' 'Glory' "A Hard Day's Night" 'Into the Wild'
 'The Barbarian Invasions' '300' 'Raging Bull' 'Amores perros'
 'Battleship Potemkin' 'The Bourne Ultimatum']
Recommendation for user:  619
MovieIds:  ['A Close Shave' 'The Godfather' 'The Philadelphia Story'
 'It Happened One Night' 'All About Eve' 'Monty Python and the Holy Grail'
 'The Wrong Trousers' 'My Left Foot: The Story of Christy Brown'
 'The Empire Strikes Back' 'Alien']
Recommendation for user:  639
MovieIds:  ['The Graduate' "Schindler's List" 'W

Recommendation for user:  646
MovieIds:  ['Jumanji' 'Copycat' 'Persuasion' 'The City of Lost Children' 'Se7en'
 'The Usual Suspects' "Mr. Holland's Opus" 'From Dusk Till Dawn'
 'Les Miserables' 'The White Balloon']
Recommendation for user:  665
MovieIds:  ['Les Miserables' 'Batman' 'The Rock' 'The Philadelphia Story'
 'My Left Foot: The Story of Christy Brown' 'The Blues Brothers'
 'Raging Bull' 'Austin Powers: International Man of Mystery'
 'A Christmas Story' 'Princess Mononoke']
Recommendation for user:  23
MovieIds:  ['Clueless' 'Searching for Bobby Fischer' 'Young Frankenstein'
 'The Breakfast Club' 'Howards End' 'The Professional'
 'Little Miss Sunshine' 'The Incredibles' 'The Red Violin'
 'The Wrong Trousers']
Recommendation for user:  30
MovieIds:  ['Richard III' 'Les Miserables' 'Dolores Claiborne'
 'Miracle on 34th Street' 'DragonHeart' 'The Nutty Professor'
 'Some Like It Hot' 'My Left Foot: The Story of Christy Brown'
 'Lawrence of Arabia' 'A Grand Day Out']
Recommendation 

Recommendation for user:  346
MovieIds:  ['The Naked Gun 33⅓: The Final Insult' 'Much Ado About Nothing' 'Rudy'
 "Singin' in the Rain" 'Star Trek: First Contact' 'As Good as It Gets'
 'Crocodile Dundee' '¡Three Amigos!' 'Body Heat' 'Man Facing Southeast']
Recommendation for user:  351
MovieIds:  ["Mr. Holland's Opus" 'Forrest Gump' 'The Lion King' 'Red Rock West'
 'Naked' 'Sleepless in Seattle' 'Aladdin' 'Cemetery Man' 'Rear Window'
 'It Happened One Night']
Recommendation for user:  353
MovieIds:  ['GoodFellas' 'Dead Poets Society' 'The Godfather: Part II'
 'Trainspotting' 'Stalag 17' 'American History X'
 'South Park: Bigger, Longer & Uncut' 'City of God' 'Leaving Las Vegas'
 'Rocky']
Recommendation for user:  358
MovieIds:  ['Les Miserables' 'The Madness of King George' 'Quiz Show'
 'Much Ado About Nothing' 'Blade Runner'
 'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb'
 'Rear Window' 'Apocalypse Now' 'Local Hero' 'The Great Escape']
Recommendation for user:  

Recommendation for user:  20
MovieIds:  ['True Romance' "Singin' in the Rain"
 'Lock, Stock and Two Smoking Barrels' 'Frequency'
 'The Twilight Saga: Breaking Dawn - Part 1' 'The Princess Bride' 'Ransom'
 'Legally Blonde' 'Cliffhanger' 'Sling Blade']
Recommendation for user:  48
MovieIds:  ['Like Water for Chocolate' 'Cinema Paradiso' 'Rain Man'
 'Terminator 2: Judgment Day' 'The African Queen' 'Casino' 'Se7en'
 'American Beauty' 'The Shawshank Redemption' 'Saving Private Ryan']
Recommendation for user:  78
MovieIds:  ['The American President' 'Four Rooms' 'The Postman' "Mr. Holland's Opus"
 'The White Balloon' "Antonia's Line" 'Bottle Rocket'
 'Anne Frank Remembered' 'Chungking Express' 'Down Periscope']
Recommendation for user:  211
MovieIds:  ['The White Balloon' 'Chungking Express' 'Love & Human Remains'
 'Eat Drink Man Woman' 'Murder in the First' 'Three Colors: Blue'
 'Red Rock West' 'The Philadelphia Story' "Singin' in the Rain"
 'Rear Window']
Recommendation for user:  236
Movi

...

Recommendation for user:  629
MovieIds:  ['Shakespeare in Love' 'Indiana Jones and the Last Crusade' 'M*A*S*H'
 'The Green Mile' 'Amadeus' 'Mary Poppins' 'The Shawshank Redemption'
 'Best in Show' 'Midnight Cowboy' 'The Intouchables']
Recommendation for user:  632
MovieIds:  ['Smoke' 'Star Wars' 'It Happened One Night' 'The African Queen'
 'Cinema Paradiso' 'Aliens' '12 Angry Men' 'Wings of Desire'
 'The Best Years of Our Lives' 'Life Is Beautiful']
Recommendation for user:  356
MovieIds:  ['Independence Day' 'Lock, Stock and Two Smoking Barrels'
 'The Incredibles' 'Iron Man' 'Raiders of the Lost Ark' 'The Dark Knight'
 'The Last of the Mohicans' 'Stardust' 'Midnight in Paris'
 'Terminator 2: Judgment Day']
Recommendation for user:  492
MovieIds:  ['Apollo 13' 'The Godfather' 'Starship Troopers' 'Ghostbusters'
 'The African Queen' 'Memento' 'Fight Club' 'I Shot Andy Warhol'
 "Hachi: A Dog's Tale" 'It Happened One Night']
Recommendation for user:  24
MovieIds:  ['The Shawshank Redemptio

MovieIds:  ['Se7en' 'The White Balloon' 'Chungking Express' 'Murder in the First'
 'Pulp Fiction' 'Three Colors: Blue' 'Clear and Present Danger'
 'Four Weddings and a Funeral' 'Short Cuts'
 'Mystery Science Theater 3000: The Movie']
Recommendation for user:  612
MovieIds:  ['The Wizard of Oz' 'The Wrong Trousers' 'The Best Years of Our Lives'
 'To Kill a Mockingbird' 'Inception' 'The Godfather'
 'Butch Cassidy and the Sundance Kid' 'Modern Times' 'Shrek'
 'A Close Shave']
Recommendation for user:  60
MovieIds:  ['The White Balloon' 'Crumb' 'Smoke' 'Leon: The Professional' 'Short Cuts'
 'The Nightmare Before Christmas' 'Terminator 2: Judgment Day' 'Emma'
 'Vertigo' 'All About Eve']
Recommendation for user:  348
MovieIds:  ['The Birdcage' 'Crimson Tide' 'Hoop Dreams' 'Clear and Present Danger'
 'Mrs. Doubtfire' 'So I Married an Axe Murderer' 'True Romance'
 'Home Alone' 'Ghost' 'Fargo']
Recommendation for user:  393
MovieIds:  ['Casablanca' 'Hoop Dreams' 'Cinema Paradiso' 'Say Anything.

Recommendation for user:  314
MovieIds:  ['A Close Shave' 'It Happened One Night' 'Sunset Boulevard'
 'The Wrong Trousers' 'Princess Mononoke' 'The Green Mile'
 'The Lord of the Rings: The Fellowship of the Ring'
 'The Lord of the Rings: The Two Towers' "All the President's Men"
 'The Lord of the Rings: The Return of the King']
Recommendation for user:  333
MovieIds:  ['The White Balloon' 'Red Rock West' 'Naked' 'Independence Day' 'Charade'
 'Wings of Desire' 'Amadeus' 'Stripes' 'Armageddon'
 'The Best Years of Our Lives']
Recommendation for user:  336
MovieIds:  ['A Christmas Story' 'The Empire Strikes Back' 'The Matrix' 'Se7en'
 'Pulp Fiction' 'Return of the Jedi' 'Cinema Paradiso'
 'The Grapes of Wrath' 'Roman Holiday' 'Trading Places']
Recommendation for user:  443
MovieIds:  ['Persuasion' 'The City of Lost Children' 'Dead Man Walking'
 'The Usual Suspects' 'Les Miserables' 'The White Balloon' 'La Haine'
 'Bottle Rocket' 'Taxi Driver' 'Anne Frank Remembered']
Recommendation for use

Recommendation for user:  332
MovieIds:  ['Leaving Las Vegas' 'Se7en' 'Leon: The Professional' 'True Romance'
 'Cinema Paradiso' 'A Clockwork Orange' "Ferris Bueller's Day Off"
 'Eternal Sunshine of the Spotless Mind' 'The Lives of Others'
 'Annie Hall']
Recommendation for user:  581
MovieIds:  ["Schindler's List" 'Indiana Jones and the Last Crusade'
 'The Dark Knight Rises' 'The Lord of the Rings: The Return of the King'
 'Avatar' 'Chariots of Fire' 'Ghost Dog: The Way of the Samurai'
 'Outbreak' 'X2' 'Gladiator']
Recommendation for user:  296
MovieIds:  ['Star Wars' 'Three Colors: Blue' 'The Lion King' 'Red Rock West' 'Speed'
 "Schindler's List" 'Sleepless in Seattle' 'Cold Comfort Farm'
 'Cemetery Man' 'The Godfather']
Recommendation for user:  600
MovieIds:  ['The Princess Bride' 'Lock, Stock and Two Smoking Barrels' 'Amélie'
 'Juno' 'Snatch' 'Donnie Darko' 'Best in Show' 'Amadeus'
 'The Intouchables' 'The Usual Suspects']
Recommendation for user:  71
MovieIds:  ['The White Balloon

Recommendation for user:  301
MovieIds:  ['Dead Man Walking' 'Apollo 13' 'Dances with Wolves'
 'The Silence of the Lambs' 'A Close Shave' '2001: A Space Odyssey'
 'Mary Poppins' 'The Wrong Trousers' "One Flew Over the Cuckoo's Nest"
 'Once Upon a Time in America']
Recommendation for user:  123
MovieIds:  ['The White Balloon' 'Anne Frank Remembered' 'Hoop Dreams'
 'Leon: The Professional' 'The Shawshank Redemption' 'Red Rock West'
 'Mrs. Doubtfire' 'Pretty Woman' 'Fargo' 'Some Like It Hot']
Recommendation for user:  258
MovieIds:  ['The Usual Suspects' 'Bottle Rocket' 'Pulp Fiction'
 'The Shawshank Redemption'
 'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb'
 'The Godfather' 'The Maltese Falcon' 'Sunset Boulevard' 'All About Eve'
 'Night of the Living Dead']
Recommendation for user:  427
MovieIds:  ['Se7en' 'The Usual Suspects' 'Braveheart' 'Taxi Driver' 'Star Wars'
 "What's Eating Gilbert Grape" 'The Piano' 'The Remains of the Day'
 "Schindler's List" 'Blade Runn

## Item-based 

In [31]:
#loading model pickle file
filename2 = 'item-based_CF.pickle'
item_cf = pickle.load(open(filename2, 'rb'))

In [32]:
# Predicting rating for the dataset
pred_rating = item_cf.test(data_testset)

In [33]:
# Saving predictions 
predictions = []
for i in range(len(pred_rating)):
    temp = pred_rating[i].est
    temp1 = round(temp,2)
    predictions.append(temp1)

In [34]:
# Converting to dataframe
predictions_df = pd.DataFrame(predictions, columns=['predictions'])

In [35]:
# Loading predictions onto dataset
item_cf_df = test_data.join(predictions_df)

In [36]:
#Dropping previous 'rating' column
item_cf_df  = item_cf_df.drop(columns=['rating'])

In [37]:
item_cf_df

,userId,movieId,predictions
0,1,1,2.82
1,1,2,2.30
2,1,3,2.10
3,1,4,1.14
4,1,5,2.16
...,...,...,...
5967518,227,161944,5.00
5967519,227,162542,5.00
5967520,227,162672,3.00
5967521,227,163056,3.54


### Printing the recommendations for Item-Based Collaborative Filtering

In [73]:
item_cf_df = pd.merge(item_cf_df, movie_title, how='left', left_on=['movieId'], right_on = ['movieId'])

In [74]:
sorted_pred = item_cf_df.sort_values(by =['userId', 'predictions'], ascending= False)

In [75]:
users = item_cf_df.userId.unique()
for i in range(len(users)):
    top10movies = sorted_pred.loc[(sorted_pred['userId'] == users[i])]['title'].values
    top10movies = top10movies[:10]
    print('Recommendation for user: ', users[i])
    print('MovieIds: ',top10movies)

Recommendation for user:  1
MovieIds:  ['Paris, France' 'Death in the Garden' "Vive L'Amour"
 'Prisoner of the Mountains' 'Dream with the Fishes'
 'Storefront Hitchcock' 'Bandits' 'Tango' 'Bedrooms and Hallways'
 'And the Ship Sails On']
Recommendation for user:  7
MovieIds:  ['Picture Bride' 'Paris, France' 'Death in the Garden' 'Angel Baby'
 'Prisoner of the Mountains' 'Dream with the Fishes' 'Gabbeh'
 'Alien Escape' 'Storefront Hitchcock' 'Bandits']
Recommendation for user:  31
MovieIds:  ['Angela' 'Love & Human Remains' 'Mute Witness' 'Picture Bride'
 'Red Firecracker, Green Firecracker' 'Paris, France' 'Faces'
 'Maya Lin: A Strong Clear Vision' 'Heavy' 'Death in the Garden']
Recommendation for user:  32
MovieIds:  ['Lamerica' 'The White Balloon' 'Anne Frank Remembered' 'Angela'
 'Love & Human Remains' 'Mute Witness' "L'Enfer" 'Picture Bride'
 'Red Firecracker, Green Firecracker' 'Paris, France']
Recommendation for user:  36
MovieIds:  ['Mute Witness' 'Picture Bride' 'Red Firecrack

Recommendation for user:  601
MovieIds:  ['Lamerica' 'Mute Witness' 'Picture Bride'
 'Red Firecracker, Green Firecracker' 'Paris, France' 'Faces'
 'Maya Lin: A Strong Clear Vision' 'Heavy' 'Death in the Garden'
 'The Day the Sun Turned Cold']
Recommendation for user:  607
MovieIds:  ['Lamerica' 'Red Firecracker, Green Firecracker' 'Paris, France' 'Faces'
 'Heavy' 'Death in the Garden' 'The Day the Sun Turned Cold'
 'Prisoner of the Mountains' 'Dream with the Fishes' 'Alien Escape']
Recommendation for user:  619
MovieIds:  ['Lamerica' 'The White Balloon' 'Mute Witness' 'Picture Bride'
 'Red Firecracker, Green Firecracker' 'Paris, France' 'Faces'
 'Maya Lin: A Strong Clear Vision' 'Heavy' 'Death in the Garden']
Recommendation for user:  639
MovieIds:  ['Picture Bride' 'Paris, France' 'Faces' 'Death in the Garden'
 'The Day the Sun Turned Cold' "Vive L'Amour"
 'Message to Love: The Isle of Wight Festival' 'Angel Baby' 'Gabbeh'
 'Tango']
Recommendation for user:  641
MovieIds:  ['Lamerica'

Recommendation for user:  577
MovieIds:  ['The Usual Suspects' 'Lamerica' 'The White Balloon'
 'Anne Frank Remembered' 'Angela' 'Amateur' 'Jeffrey'
 'Love & Human Remains' 'Mute Witness' 'Smoke']
Recommendation for user:  582
MovieIds:  ['Lamerica' 'Picture Bride' 'Red Firecracker, Green Firecracker'
 'Paris, France' 'Heavy' 'Death in the Garden' 'Project S'
 'Little Lord Fauntleroy' 'Angel Baby' 'Hotel de Love']
Recommendation for user:  602
MovieIds:  ['Mute Witness' 'Picture Bride' 'Faces' 'Maya Lin: A Strong Clear Vision'
 'The Day the Sun Turned Cold' "Vive L'Amour" 'Little Lord Fauntleroy'
 'Female Perversions' 'Message to Love: The Isle of Wight Festival'
 'Angel Baby']
Recommendation for user:  603
MovieIds:  ['Lamerica' 'The White Balloon' 'Anne Frank Remembered'
 'Love & Human Remains' 'Mute Witness' "L'Enfer" 'Picture Bride'
 'Red Firecracker, Green Firecracker' 'Paris, France' 'Land and Freedom']
Recommendation for user:  605
MovieIds:  ['Prisoner of the Mountains' 'Dream w

Recommendation for user:  195
MovieIds:  ['Paris, France' 'Death in the Garden' "The Mummy's Ghost"
 'House of Frankenstein' 'Son of Frankenstein' 'The Ghost of Frankenstein'
 'Prisoner of the Mountains' 'Dream with the Fishes'
 'Storefront Hitchcock' 'Two Ninas']
Recommendation for user:  248
MovieIds:  ['Lamerica' 'The White Balloon' 'Anne Frank Remembered' 'Angela'
 'Love & Human Remains' 'Mute Witness' 'Nadja' "L'Enfer" 'Little Odessa'
 'Picture Bride']
Recommendation for user:  276
MovieIds:  ['Lamerica' 'Mute Witness' 'Picture Bride' 'Paris, France' 'Faces'
 'Death in the Garden' 'The Day the Sun Turned Cold' "Vive L'Amour"
 'Little Lord Fauntleroy' 'Female Perversions']
Recommendation for user:  299
MovieIds:  ['Lamerica' 'Anne Frank Remembered' 'Angela' 'Mute Witness' "L'Enfer"
 'Picture Bride' 'Red Firecracker, Green Firecracker'
 'The Shawshank Redemption'
 'The Wonderful, Horrible Life of Leni Riefenstahl' 'Paris, France']
Recommendation for user:  346
MovieIds:  ['Prisoner 

Recommendation for user:  391
MovieIds:  ['Paris, France' 'Heavy' 'Death in the Garden'
 'The Day the Sun Turned Cold' 'Prisoner of the Mountains'
 'Dream with the Fishes' 'Alien Escape' 'Storefront Hitchcock' 'Tango'
 'Endurance']
Recommendation for user:  430
MovieIds:  ['Lamerica' 'Mute Witness' 'Picture Bride'
 'Red Firecracker, Green Firecracker' 'Faces'
 'Maya Lin: A Strong Clear Vision' 'Heavy' 'The Day the Sun Turned Cold'
 "Vive L'Amour" 'Project S']
Recommendation for user:  441
MovieIds:  ['Paris, France' 'Faces' 'Death in the Garden'
 'Message to Love: The Isle of Wight Festival' 'Prisoner of the Mountains'
 'Dream with the Fishes' 'Storefront Hitchcock' 'Bandit Queen' 'Tango'
 'Bedrooms and Hallways']
Recommendation for user:  460
MovieIds:  ['Lamerica' 'Picture Bride' "Vive L'Amour" 'Project S'
 'Little Lord Fauntleroy' 'Message to Love: The Isle of Wight Festival'
 'Angel Baby' 'Hotel de Love' 'Gabbeh' 'Taste of Cherry']
Recommendation for user:  481
MovieIds:  ['Lameric

...

Recommendation for user:  63
MovieIds:  ['Lamerica' 'Mute Witness' "L'Enfer" 'Red Firecracker, Green Firecracker'
 'Paris, France' 'Faces' 'Maya Lin: A Strong Clear Vision'
 'Death in the Garden' "Vive L'Amour" 'Project S']
Recommendation for user:  142
MovieIds:  ['Paris, France' 'Death in the Garden' 'The Art of War' 'The Circle'
 "Bang Bang You're Dead" 'Dark Days' 'Law of Desire' 'Inside Deep Throat'
 'Kids in America' 'State of Siege']
Recommendation for user:  146
MovieIds:  ['Lamerica' 'Picture Bride' 'Paris, France' 'Heavy' 'Death in the Garden'
 'The Day the Sun Turned Cold' 'Project S' 'Little Lord Fauntleroy'
 'Message to Love: The Isle of Wight Festival' 'Angel Baby']
Recommendation for user:  273
MovieIds:  ['Lamerica' 'The White Balloon' 'Anne Frank Remembered' 'Angela'
 'Love & Human Remains' 'Mute Witness'
 'Red Firecracker, Green Firecracker'
 'The Wonderful, Horrible Life of Leni Riefenstahl' 'Paris, France'
 'Faces']
Recommendation for user:  286
MovieIds:  ['Lameric

Recommendation for user:  644
MovieIds:  ['Lamerica' 'Anne Frank Remembered' 'Love & Human Remains' 'Mute Witness'
 "L'Enfer" 'Little Odessa' 'Picture Bride'
 'Red Firecracker, Green Firecracker' 'Paris, France' 'Land and Freedom']
Recommendation for user:  12
MovieIds:  ['Prisoner of the Mountains' 'Dream with the Fishes'
 'Storefront Hitchcock' 'Bandits' "The Mummy's Ghost"
 'House of Frankenstein' 'Son of Frankenstein' 'The Ghost of Frankenstein'
 'Vampyros Lesbos' 'Defying Gravity']
Recommendation for user:  189
MovieIds:  ['Lamerica' 'Paris, France' 'The Promise' 'Faces'
 'Maya Lin: A Strong Clear Vision' 'Heavy'
 'My Life and Times With Antonin Artaud' 'Death in the Garden'
 'The Ballad of Narayama' "Vive L'Amour"]
Recommendation for user:  215
MovieIds:  ['Paris, France' 'Death in the Garden' 'Prisoner of the Mountains'
 'Dream with the Fishes' 'Alien Escape' 'Storefront Hitchcock' 'Tango'
 'And the Ship Sails On' 'The Best Man' 'Brother Can You Spare A Dime']
Recommendation for

Recommendation for user:  630
MovieIds:  ['Mute Witness' 'Red Firecracker, Green Firecracker' 'Paris, France'
 'Death in the Garden' "Vive L'Amour" 'Little Lord Fauntleroy'
 'Female Perversions' 'Message to Love: The Isle of Wight Festival'
 'Hotel de Love' 'Losing Chase']
Recommendation for user:  660
MovieIds:  ['Paris, France' 'Faces' 'Death in the Garden' "Vive L'Amour"
 'Paths of Glory' 'Ran' 'Taste of Cherry' 'Character' 'Hav Plenty'
 "The Mummy's Ghost"]
Recommendation for user:  663
MovieIds:  ['Paris, France' 'Death in the Garden' 'The Day the Sun Turned Cold'
 "Vive L'Amour" 'Prisoner of the Mountains' 'Dream with the Fishes'
 'Alien Escape' 'Storefront Hitchcock' 'Taste of Cherry' 'Tango']
Recommendation for user:  52
MovieIds:  ['Lamerica' 'Love & Human Remains' "L'Enfer" 'Paris, France' 'Faces'
 'Maya Lin: A Strong Clear Vision' 'Heavy' 'Death in the Garden'
 'The Day the Sun Turned Cold' 'Cyclo']
Recommendation for user:  498
MovieIds:  ['Paris, France' 'Death in the Gard

Recommendation for user:  429
MovieIds:  ['Lamerica' 'Picture Bride' 'Heavy' 'Angel Baby'
 'Prisoner of the Mountains' 'Dream with the Fishes' 'Gabbeh'
 'Storefront Hitchcock' 'One Magic Christmas' "The Mummy's Ghost"]
Recommendation for user:  476
MovieIds:  ["L'Enfer" 'Paris, France' 'Death in the Garden' "Vive L'Amour" 'Tango'
 'Bedrooms and Hallways' 'And the Ship Sails On'
 'Brother Can You Spare A Dime' 'Defying Gravity' 'Lonely Are the Brave']
Recommendation for user:  558
MovieIds:  ['Lamerica' 'The White Balloon' 'Anne Frank Remembered' 'Angela'
 'Love & Human Remains' 'Mute Witness' 'Nadja' 'Little Odessa'
 'Picture Bride' 'Red Firecracker, Green Firecracker']
Recommendation for user:  622
MovieIds:  ['The Usual Suspects' 'Lamerica' 'The White Balloon' 'Taxi Driver'
 'Anne Frank Remembered' 'Angela' 'Amateur' 'Jeffrey'
 'Love & Human Remains' 'Mute Witness']
Recommendation for user:  170
MovieIds:  ["L'Enfer" 'Picture Bride' 'Paris, France' 'The Promise'
 'My Life and Times W

Recommendation for user:  526
MovieIds:  ['Lamerica' 'Anne Frank Remembered' 'Angela' 'Mute Witness'
 'Picture Bride' 'Red Firecracker, Green Firecracker' 'Paris, France'
 'Faces' 'Maya Lin: A Strong Clear Vision' 'Heavy']
Recommendation for user:  204
MovieIds:  ['Paris, France' 'Heavy' 'Death in the Garden' "Vive L'Amour"
 'Prisoner of the Mountains' 'Dream with the Fishes'
 'Storefront Hitchcock' 'Bandits' 'Brother Can You Spare A Dime'
 'Defying Gravity']
Recommendation for user:  221
MovieIds:  ['Paris, France' 'Heavy' 'Death in the Garden'
 'The Day the Sun Turned Cold' "Vive L'Amour" 'Prisoner of the Mountains'
 'Dream with the Fishes' 'Storefront Hitchcock' 'Bandits' 'Tango']
Recommendation for user:  45
MovieIds:  ['Lamerica' 'Angela' 'Red Firecracker, Green Firecracker' 'Paris, France'
 'Heavy' 'Death in the Garden' 'The Day the Sun Turned Cold'
 "Vive L'Amour" 'Little Lord Fauntleroy'
 'Message to Love: The Isle of Wight Festival']
Recommendation for user:  289
MovieIds:  ['

### User-based

In [40]:
#loading model pickle file
filename3 = 'user-based_CF.pickle'
item_cf = pickle.load(open(filename3, 'rb'))

In [41]:
# Predicting rating for the dataset
pred_rating = item_cf.test(data_testset)

In [42]:
# Saving predictions 
predictions = []
for i in range(len(pred_rating)):
    temp = pred_rating[i].est
    temp1 = round(temp,2)
    predictions.append(temp1)

In [43]:
# Converting to dataframe
predictions_df = pd.DataFrame(predictions, columns=['predictions'])

In [44]:
# Loading predictions onto dataset
user_cf_df = test_data.join(predictions_df)

In [45]:
#Dropping previous 'rating' column
user_cf_df  = user_cf_df.drop(columns=['rating'])

In [46]:
user_cf_df

,userId,movieId,predictions
0,1,1,2.69
1,1,2,2.20
2,1,3,1.86
3,1,4,1.32
4,1,5,2.08
...,...,...,...
5967518,227,161944,4.29
5967519,227,162542,3.81
5967520,227,162672,3.81
5967521,227,163056,3.54


### Printing the recommendations for User-Based Collaborative Filtering

In [76]:
user_cf_df = pd.merge(user_cf_df, movie_title, how='left', left_on=['movieId'], right_on = ['movieId'])

In [77]:
sorted_pred = user_cf_df.sort_values(by =['userId', 'predictions'], ascending= False)

In [78]:
users = user_cf_df.userId.unique()
for i in range(len(users)):
    top10movies = sorted_pred.loc[(sorted_pred['userId'] == users[i])]['title'].values
    top10movies = top10movies[:10]
    print('Recommendation for user: ', users[i])
    print('MovieIds: ',top10movies)

Recommendation for user:  1
MovieIds:  ['Vampyros Lesbos' 'Dylan Moran: Monster' 'The King Is Alive' 'Innocence'
 'Maelström' 'Cool as Ice' 'The Looney, Looney, Looney Bugs Bunny Movie'
 'Love Me If You Dare' 'Seve' 'Land of Silence and Darkness']
Recommendation for user:  7
MovieIds:  ['Cronos' 'The Day the Sun Turned Cold' 'Prisoner of the Mountains'
 'Dream with the Fishes' 'Storefront Hitchcock' 'Taste of Cherry'
 'Vampyros Lesbos' 'Cool as Ice' 'The Art of War' 'Two Ninas']
Recommendation for user:  31
MovieIds:  ['Lamerica' 'The White Balloon' 'Anne Frank Remembered'
 'Love & Human Remains' 'Mute Witness' 'Nadja' 'Picture Bride'
 'Red Firecracker, Green Firecracker'
 'The Wonderful, Horrible Life of Leni Riefenstahl'
 'Faster, Pussycat! Kill! Kill!']
Recommendation for user:  32
MovieIds:  ['Lamerica' 'Mute Witness' 'Picture Bride' 'Cronos'
 'Maya Lin: A Strong Clear Vision' 'The Day the Sun Turned Cold'
 'Project S' 'Female Perversions' 'Angel Baby' 'Prisoner of the Mountains']


Recommendation for user:  601
MovieIds:  ['Mute Witness' 'Double Happiness' 'Cronos' 'Project S'
 'Female Perversions' 'Prisoner of the Mountains' 'That Old Feeling'
 'Losing Chase' 'The Designated Mourner' 'Dream with the Fishes']
Recommendation for user:  607
MovieIds:  ['Lamerica' 'Mute Witness' 'Picture Bride' 'Paris, France' 'Cronos'
 'Maya Lin: A Strong Clear Vision' 'Death in the Garden'
 'The Day the Sun Turned Cold' 'Project S' 'Female Perversions']
Recommendation for user:  619
MovieIds:  ['Picture Bride' 'Cronos' 'The Day the Sun Turned Cold' 'Project S'
 'Angel Baby' 'Prisoner of the Mountains' 'Dream with the Fishes' 'Gabbeh'
 'Storefront Hitchcock' 'Taste of Cherry']
Recommendation for user:  639
MovieIds:  ['Cronos' 'The Day the Sun Turned Cold' 'Prisoner of the Mountains'
 'Dream with the Fishes' 'Storefront Hitchcock' 'Taste of Cherry'
 'Vampyros Lesbos' 'Cool as Ice' 'The Art of War' 'Two Ninas']
Recommendation for user:  641
MovieIds:  ['Lamerica' 'Mute Witness' 'Pic

Recommendation for user:  605
MovieIds:  ['Prisoner of the Mountains' 'Dream with the Fishes'
 'Storefront Hitchcock' 'Vampyros Lesbos' 'Cool as Ice' 'The Art of War'
 'Two Ninas' 'The King Is Alive' 'Erik the Viking' 'Innocence']
Recommendation for user:  646
MovieIds:  ['Lamerica' 'The White Balloon' 'Anne Frank Remembered'
 'The Amazing Panda Adventure' 'Mute Witness' 'Nadja' "L'Enfer"
 'Picture Bride' 'Red Firecracker, Green Firecracker'
 'The Wonderful, Horrible Life of Leni Riefenstahl']
Recommendation for user:  665
MovieIds:  ['The Day the Sun Turned Cold' 'Prisoner of the Mountains'
 'Dream with the Fishes' 'Storefront Hitchcock' 'Vampyros Lesbos'
 'Cool as Ice' 'The Art of War' 'Two Ninas' 'The King Is Alive'
 'Erik the Viking']
Recommendation for user:  23
MovieIds:  ['Lamerica' 'Mute Witness' 'Picture Bride' 'Paris, France' 'Cronos'
 'Maya Lin: A Strong Clear Vision' 'Death in the Garden'
 'The Day the Sun Turned Cold' 'Project S' 'Female Perversions']
Recommendation for us

Recommendation for user:  195
MovieIds:  ['Prisoner of the Mountains' 'Dream with the Fishes'
 'Storefront Hitchcock' 'Vampyros Lesbos' 'Cool as Ice' 'The Art of War'
 'Two Ninas' 'The King Is Alive' 'Erik the Viking' 'Innocence']
Recommendation for user:  248
MovieIds:  ['Lamerica' 'The White Balloon' 'Last Summer in the Hamptons'
 'Anne Frank Remembered' 'The Amazing Panda Adventure'
 'Love & Human Remains' 'Mute Witness' 'Nadja' 'Exotica' 'Picture Bride']
Recommendation for user:  276
MovieIds:  ['Vampyros Lesbos' 'Cool as Ice' 'The Art of War' 'The King Is Alive'
 'Innocence' 'Maelström' 'The Looney, Looney, Looney Bugs Bunny Movie'
 'Land of Silence and Darkness' 'The Goat' 'The Play House']
Recommendation for user:  299
MovieIds:  ['Lamerica' 'The White Balloon' 'Anne Frank Remembered'
 'The Amazing Panda Adventure' 'Mute Witness' 'Nadja' 'Picture Bride'
 'Red Firecracker, Green Firecracker'
 'The Wonderful, Horrible Life of Leni Riefenstahl'
 'Faster, Pussycat! Kill! Kill!']
Rec

Recommendation for user:  460
MovieIds:  ['Lamerica' 'Mute Witness' 'Picture Bride' 'Paris, France' 'Cronos'
 'Maya Lin: A Strong Clear Vision' 'Heavy' 'Death in the Garden'
 'The Day the Sun Turned Cold' 'Project S']
Recommendation for user:  481
MovieIds:  ['Lamerica' 'The White Balloon' 'Anne Frank Remembered'
 'The Amazing Panda Adventure' 'Mute Witness' 'Nadja' 'Picture Bride'
 'Red Firecracker, Green Firecracker'
 'The Wonderful, Horrible Life of Leni Riefenstahl'
 'Faster, Pussycat! Kill! Kill!']
Recommendation for user:  497
MovieIds:  ['Lamerica' 'Mute Witness' 'Picture Bride' 'Paris, France' 'Cronos'
 'Maya Lin: A Strong Clear Vision' 'Heavy' 'Walking and Talking'
 'Death in the Garden' 'The Day the Sun Turned Cold']
Recommendation for user:  505
MovieIds:  ['The Day the Sun Turned Cold' 'Prisoner of the Mountains'
 'Dream with the Fishes' 'Storefront Hitchcock' 'Vampyros Lesbos'
 'Cool as Ice' 'The Art of War' 'Two Ninas' 'The King Is Alive'
 'Erik the Viking']
Recommendatio

...

Recommendation for user:  317
MovieIds:  ['Cronos' 'The Day the Sun Turned Cold' 'Prisoner of the Mountains'
 'Dream with the Fishes' 'Storefront Hitchcock' 'Taste of Cherry'
 'Man Facing Southeast' 'Vampyros Lesbos' 'Cool as Ice' 'Two Ninas']
Recommendation for user:  365
MovieIds:  ['Lamerica' 'The White Balloon' 'Anne Frank Remembered' 'Amateur'
 'Mute Witness' 'Nadja' 'Picture Bride'
 'Red Firecracker, Green Firecracker'
 'The Wonderful, Horrible Life of Leni Riefenstahl'
 'Faster, Pussycat! Kill! Kill!']
Recommendation for user:  402
MovieIds:  ['Lamerica' 'Mute Witness' 'Picture Bride' 'Cronos'
 'Maya Lin: A Strong Clear Vision' 'The Day the Sun Turned Cold'
 'Project S' 'Female Perversions' 'Angel Baby' 'Prisoner of the Mountains']
Recommendation for user:  437
MovieIds:  ['Lamerica' 'Mute Witness' 'Picture Bride' 'Cronos'
 'Maya Lin: A Strong Clear Vision' 'Heavy' 'The Day the Sun Turned Cold'
 'Project S' 'Female Perversions' 'Hearts and Minds']
Recommendation for user:  479
M

Recommendation for user:  368
MovieIds:  ['Mute Witness' 'Picture Bride' 'Paris, France' 'Cronos'
 'Death in the Garden' 'The Day the Sun Turned Cold' 'Project S'
 'Female Perversions' 'Angel Baby' 'Prisoner of the Mountains']
Recommendation for user:  645
MovieIds:  ['Lamerica' 'Mute Witness' 'Red Firecracker, Green Firecracker'
 'Double Happiness' 'My Favorite Season' 'Maya Lin: A Strong Clear Vision'
 'Heavy' 'The Day the Sun Turned Cold' 'Project S' 'Female Perversions']
Recommendation for user:  515
MovieIds:  ['Lamerica' 'The Confessional' 'The White Balloon' 'Unforgettable'
 'Anne Frank Remembered' 'The Amazing Panda Adventure' 'Amateur'
 'Mute Witness' 'Nadja' 'Exotica']
Recommendation for user:  107
MovieIds:  ['Prisoner of the Mountains' 'Dream with the Fishes'
 'Storefront Hitchcock' 'Vampyros Lesbos' 'Cool as Ice' 'The Art of War'
 'Two Ninas' 'The King Is Alive' 'Erik the Viking' 'Innocence']
Recommendation for user:  112
MovieIds:  ['Lamerica' 'The White Balloon' 'Anne Fr

Recommendation for user:  630
MovieIds:  ['Lamerica' 'The White Balloon' 'Anne Frank Remembered'
 'The Amazing Panda Adventure' 'Mute Witness' 'Nadja' "L'Enfer"
 'Picture Bride' 'Red Firecracker, Green Firecracker'
 'The Wonderful, Horrible Life of Leni Riefenstahl']
Recommendation for user:  660
MovieIds:  ['Lamerica' 'The White Balloon' 'Anne Frank Remembered'
 'The Amazing Panda Adventure' 'Mute Witness' 'Nadja' 'Exotica' "L'Enfer"
 'Picture Bride' 'Red Firecracker, Green Firecracker']
Recommendation for user:  663
MovieIds:  ['Lamerica' 'Mute Witness' 'Picture Bride' 'Double Happiness'
 'Maya Lin: A Strong Clear Vision' 'The Day the Sun Turned Cold'
 'Project S' 'Jude' 'Female Perversions' 'Angel Baby']
Recommendation for user:  52
MovieIds:  ['Lamerica' 'Mute Witness' 'Nadja' 'Hideaway' 'Picture Bride'
 'Paris, France' 'Maya Lin: A Strong Clear Vision' 'Heavy'
 'Death in the Garden' 'The Day the Sun Turned Cold']
Recommendation for user:  498
MovieIds:  ['Prisoner of the Mountains

Recommendation for user:  429
MovieIds:  ['Death of a Superhero' 'Vampyros Lesbos' 'Dylan Moran: Monster'
 'The King Is Alive' 'Innocence' 'Maelström' 'Incendies' 'Cool as Ice'
 'The Looney, Looney, Looney Bugs Bunny Movie' 'Seve']
Recommendation for user:  476
MovieIds:  ['Prisoner of the Mountains' 'Dream with the Fishes'
 'Storefront Hitchcock' 'Vampyros Lesbos' 'Cool as Ice' 'Two Ninas'
 'The King Is Alive' 'Erik the Viking' 'Innocence' 'The Grass Is Greener']
Recommendation for user:  558
MovieIds:  ['Lamerica' 'The White Balloon' 'Last Summer in the Hamptons' 'La Haine'
 'Anne Frank Remembered' 'The Amazing Panda Adventure'
 'Love & Human Remains' 'Mute Witness' 'Nadja' 'Exotica']
Recommendation for user:  622
MovieIds:  ['Toy Story' 'Heat' 'Balto' 'Casino' 'Sense and Sensibility' 'Othello'
 'Persuasion' 'The City of Lost Children' 'Shanghai Triad'
 'Twelve Monkeys']
Recommendation for user:  170
MovieIds:  ['Death of a Superhero' 'Vampyros Lesbos' 'Dylan Moran: Monster'
 'The Ki

Recommendation for user:  526
MovieIds:  ['Heidi Fleiss: Hollywood Madam' 'Unforgettable' 'Anne Frank Remembered'
 'Amateur' 'Mute Witness' 'Nadja' 'Safe' 'Exotica'
 'Red Firecracker, Green Firecracker' 'Double Happiness']
Recommendation for user:  204
MovieIds:  ['The Day the Sun Turned Cold' 'Prisoner of the Mountains'
 'Dream with the Fishes' 'Storefront Hitchcock' 'Taste of Cherry'
 'Vampyros Lesbos' 'Cool as Ice' 'The Art of War' 'Two Ninas'
 'The King Is Alive']
Recommendation for user:  221
MovieIds:  ['Vampyros Lesbos' 'The Art of War' 'The King Is Alive' 'Innocence'
 'Maelström' 'Incendies' 'Dylan Moran: Monster' 'Cool as Ice'
 'The Looney, Looney, Looney Bugs Bunny Movie' 'Seve']
Recommendation for user:  45
MovieIds:  ['Anne Frank Remembered' 'Mute Witness' 'Picture Bride'
 'The Day the Sun Turned Cold' 'Project S' 'Female Perversions'
 'Angel Baby' 'That Old Feeling' 'Losing Chase' 'The Designated Mourner']
Recommendation for user:  289
MovieIds:  ['Vampyros Lesbos' 'Cool a

# Hybrid Models

# Hybrid Approach A (Content-Based and Item-Based Collaborative Filter)

In [79]:

df = pd.DataFrame()
df = pd.merge(item_cf_df, recom_content_pred, how= 'inner', left_on=['userId','movieId'], right_on = ['userId','movieId'])

In [80]:
df_test = pd.DataFrame()
df_test['predictions'] = df['predictions']
df_test['prediction'] = df['prediction']
x_test = df_test.values

In [81]:
with open('hybrid_collaborative-item_content.pkl', 'rb') as file:
    pickle_model = pickle.load(file)
    
Ypredict = pickle_model.predict(x_test)

In [82]:
df['hybrid_pred'] =  Ypredict 

In [83]:
df.head()

,userId,movieId,predictions,title_x,prediction,title_y,hybrid_pred
0,1,1,2.82,Toy Story,3.538545,Toy Story,3.501551
1,1,2,2.30,Jumanji,3.538545,Jumanji,3.501551
2,1,3,2.10,Grumpier Old Men,3.538545,Grumpier Old Men,3.501551
3,1,4,1.14,Waiting to Exhale,3.538545,Waiting to Exhale,3.501551
4,1,5,2.16,Father of the Bride Part II,3.538545,Father of the Bride Part II,3.501551


### Printing the recommendations for Hybrid Approach A

In [84]:
df= pd.merge(df, movie_title, how='left', left_on=['movieId'], right_on = ['movieId'])

In [85]:
sorted_pred = df.sort_values(by =['userId', 'hybrid_pred'], ascending= False)

In [86]:
users = df.userId.unique()
for i in range(len(users)):
    top10movies = sorted_pred.loc[(sorted_pred['userId'] == users[i])]['title'].values
    top10movies = top10movies[:10]
    print('Recommendation for user: ', users[i])
    print('MovieIds: ',top10movies)

Recommendation for user:  1
MovieIds:  ['Stealth' 'Toy Story' 'Jumanji' 'Grumpier Old Men' 'Waiting to Exhale'
 'Father of the Bride Part II' 'Heat' 'Sabrina' 'Tom and Huck'
 'Sudden Death']
Recommendation for user:  7
MovieIds:  ['Jumanji' 'Grumpier Old Men' 'Waiting to Exhale'
 'Father of the Bride Part II' 'Sabrina' 'Sudden Death'
 'The American President' 'Dracula: Dead and Loving It' 'Nixon'
 'Cutthroat Island']
Recommendation for user:  31
MovieIds:  ['My Super Ex-Girlfriend' 'Jason and the Argonauts' 'Jack & Sarah'
 'The Bridges of Madison County' 'Blood, Guts, Bullets and Octane'
 'Only the Strong' 'Suicide Club' 'The Whistleblower' 'Sex Ed'
 'People, Places, Things']
Recommendation for user:  32
MovieIds:  ['Maverick' 'D3: The Mighty Ducks' 'eXistenZ' 'Any Given Sunday'
 'Panic Room' 'Spy Kids 3-D: Game Over' 'The Brothers Grimm'
 'Zathura: A Space Adventure' 'Gamer' 'Ouija']
Recommendation for user:  36
MovieIds:  ['Star Trek V: The Final Frontier' 'Biloxi Blues' 'İtirazım Va

Recommendation for user:  639
MovieIds:  ['Senior Trip' 'Shaft' "Don't Tell Mom the Babysitter's Dead" 'Suspect'
 'Kansas' 'Sleuth' 'Georgia' 'Mildred Pierce' '3:10 to Yuma'
 '3:10 to Yuma']
Recommendation for user:  641
MovieIds:  ['Mask' 'Dirty Work' 'Indecent Proposal' "What's Up, Tiger Lily?"
 'For Your Consideration' 'Malice' "White Men Can't Jump" 'Match Point'
 'The Messenger' 'Rebel Without a Cause']
Recommendation for user:  19
MovieIds:  ['Mask' "The Ballad of Ramblin' Jack" 'The Thin Red Line'
 'Goodbye, Columbus' 'Rudolph, the Red-Nosed Reindeer' 'Malice'
 'A Midnight Clear' 'For Your Consideration' 'Father of the Bride Part II'
 'Dracula: Dead and Loving It']
Recommendation for user:  35
MovieIds:  ['Toy Story' 'Jumanji' 'Grumpier Old Men' 'Waiting to Exhale'
 'Father of the Bride Part II' 'Heat' 'Sabrina' 'Tom and Huck'
 'Sudden Death' 'GoldenEye']
Recommendation for user:  61
MovieIds:  ['The Brothers Grimm' 'Zathura: A Space Adventure' 'Toy Story'
 'Grumpier Old Men' 'W

Recommendation for user:  70
MovieIds:  ['Malice' 'Rebel Without a Cause' 'Condorman' 'Indecent Proposal'
 'Radio Days' 'The Funhouse' 'Toy Story 2' "White Men Can't Jump"
 'Bound for Glory' 'Hollywood Ending']
Recommendation for user:  90
MovieIds:  ['Condorman' 'Indecent Proposal' "What's Up, Tiger Lily?" 'Africa Screams'
 'For Your Consideration' 'The Messenger' 'Malice' 'Radio Days'
 'Toy Story 2' "White Men Can't Jump"]
Recommendation for user:  105
MovieIds:  ['Mulholland Falls' 'Death Wish 2' 'The Crush' 'Escape from L.A.'
 'Biloxi Blues' 'The Cincinnati Kid' 'Benny & Joon' 'Now You See Me'
 'Perfect Stranger' 'Jumanji']
Recommendation for user:  119
MovieIds:  ['A Midnight Clear' 'Goodbye, Columbus' 'The Thin Red Line'
 'Tai-Chi Master' 'Love! Valour! Compassion!' 'Mask'
 'For Your Consideration' 'Rudolph, the Red-Nosed Reindeer' 'Dirty Work'
 'Stanley Kubrick: A Life in Pictures']
Recommendation for user:  141
MovieIds:  ['The Getaway' 'No Escape' 'Cool Hand Luke' 'American Gi

Recommendation for user:  423
MovieIds:  ['Donnie Brasco' 'Things Change' 'İtirazım Var'
 'Ghost Dog: The Way of the Samurai' 'Think Like a Man Too'
 "Jane Austen's Mafia!" 'Bad Lieutenant' 'The Electric Horseman' 'Bugsy'
 'Tracers']
Recommendation for user:  463
MovieIds:  ['Rudolph, the Red-Nosed Reindeer' 'The Thin Red Line' 'Mask'
 'A Midnight Clear' 'Let It Ride' "Max Keeble's Big Move"
 'Goodbye, Columbus' 'Room' 'Jumanji' 'Grumpier Old Men']
Recommendation for user:  522
MovieIds:  ['BASEketball' 'Stay Alive' 'Toy Story' 'Grumpier Old Men'
 'Waiting to Exhale' 'Father of the Bride Part II' 'Sabrina'
 'Tom and Huck' 'Sudden Death' 'GoldenEye']
Recommendation for user:  580
MovieIds:  ['Smiles of a Summer Night' 'Malice' 'The Thin Red Line'
 "What's Up, Tiger Lily?" 'Room' 'The Funhouse' 'Rebel Without a Cause'
 'Condorman' 'Indecent Proposal' 'Match Point']
Recommendation for user:  585
MovieIds:  ['Stepmom' 'Hollywood Ending' 'A Midnight Clear'
 'Smiles of a Summer Night' 'Goodb

Recommendation for user:  17
MovieIds:  ['Star Trek V: The Final Frontier' 'The Color of Money' 'Falling Down'
 'Twins' 'Vincent & Theo' 'City of Hope' 'Vincent' 'The Reader' 'The Town'
 'Beautiful Thing']
Recommendation for user:  22
MovieIds:  ['Kafka' 'I, Robot' 'Æon Flux' 'The Bucket List' 'Eye of the Beholder'
 "Big Momma's House" 'The Glimmer Man' 'This Island Earth'
 'Cruel Intentions' '28 Weeks Later']
Recommendation for user:  152
MovieIds:  ['Stepmom' "Max Keeble's Big Move" 'A Midnight Clear' 'Tai-Chi Master'
 'The Thin Red Line' 'Rebel Without a Cause' 'Radio Days' 'Condorman'
 'Jumanji' 'Grumpier Old Men']
Recommendation for user:  257
MovieIds:  ['Toy Story' 'Jumanji' 'Grumpier Old Men' 'Waiting to Exhale'
 'Father of the Bride Part II' 'Heat' 'Sabrina' 'Sudden Death' 'GoldenEye'
 'The American President']
Recommendation for user:  265
MovieIds:  ['Muppet Treasure Island' 'S1m0ne' '42nd Street' 'The Artist'
 'Straight-Jacket' 'Men with Guns' 'Trouble in Paradise'
 'What J

...

Recommendation for user:  628
MovieIds:  ['Diamonds Are Forever' 'Song of the Thin Man' 'Toy Story' 'Jumanji'
 'Grumpier Old Men' 'Waiting to Exhale' 'Father of the Bride Part II'
 'Heat' 'Sabrina' 'Tom and Huck']
Recommendation for user:  629
MovieIds:  ['Jumanji' 'Grumpier Old Men' 'Waiting to Exhale'
 'Father of the Bride Part II' 'Sabrina' 'Sudden Death' 'GoldenEye'
 'The American President' 'Dracula: Dead and Loving It' 'Nixon']
Recommendation for user:  632
MovieIds:  ['Swingers' 'Night of the Living Dead' 'Waiting to Exhale'
 'Dracula: Dead and Loving It' 'Cutthroat Island'
 'Ace Ventura: When Nature Calls' 'Money Train' 'Across the Sea of Time'
 'It Takes Two' 'Dead Presidents']
Recommendation for user:  356
MovieIds:  ['Jumanji' 'Grumpier Old Men' 'Waiting to Exhale'
 'Father of the Bride Part II' 'Sabrina' 'Sudden Death' 'GoldenEye'
 'The American President' 'Dracula: Dead and Loving It' 'Nixon']
Recommendation for user:  492
MovieIds:  ['One from the Heart' 'Nine 1/2 Weeks' 

Recommendation for user:  60
MovieIds:  ['Leaving Las Vegas' 'Showgirls' 'Go' 'Funny Bones' 'Black Rain' 'Bugsy'
 'Last Vegas' 'Think Like a Man Too' 'Little Odessa' 'Donnie Brasco']
Recommendation for user:  348
MovieIds:  ['Caught' 'Carrie' 'The Butterfly Effect 3: Revelations'
 'Young Sherlock Holmes' 'The Serpent and the Rainbow' 'Jumanji'
 'Grumpier Old Men' 'Waiting to Exhale' 'Father of the Bride Part II'
 'Sabrina']
Recommendation for user:  393
MovieIds:  ['Funny Bones' 'Jumanji' 'Grumpier Old Men' 'Waiting to Exhale'
 'Father of the Bride Part II' 'Sabrina' 'Tom and Huck' 'Sudden Death'
 'GoldenEye' 'The American President']
Recommendation for user:  644
MovieIds:  ['The Postman Always Rings Twice' 'The Wizard of Oz' 'Edge of Seventeen'
 'Grumpier Old Men' 'Father of the Bride Part II' 'Sabrina' 'Sudden Death'
 'Dracula: Dead and Loving It' 'Cutthroat Island' 'Four Rooms']
Recommendation for user:  12
MovieIds:  ['Deep Red' 'Toy Story' 'Jumanji' 'Grumpier Old Men' 'Waiting to

Recommendation for user:  443
MovieIds:  ['The Swan Princess' 'Hot Shots! Part Deux'
 'Star Wars: Episode I - The Phantom Menace' 'The 7th Voyage of Sinbad'
 'Shanghai Noon' 'The Thief of Bagdad' 'The Hidden Fortress'
 'Star Wars: Episode III - Revenge of the Sith'
 'Star Wars: The Force Awakens' 'Kingsglaive: Final Fantasy XV']
Recommendation for user:  448
MovieIds:  ['Rebel Without a Cause' 'Waiting to Exhale' 'Cutthroat Island'
 'Ace Ventura: When Nature Calls' 'Money Train' 'Across the Sea of Time'
 'It Takes Two' 'Dead Presidents' 'Mortal Kombat' 'Two If by Sea']
Recommendation for user:  456
MovieIds:  ['Goodbye, Columbus' 'Mask' 'Rebel Without a Cause' 'Jumanji'
 'Grumpier Old Men' 'Waiting to Exhale' 'Father of the Bride Part II'
 'Sabrina' 'Sudden Death' 'GoldenEye']
Recommendation for user:  486
MovieIds:  ['Malice' 'Condorman' 'Indecent Proposal' 'Radio Days' 'The Funhouse'
 'Bound for Glory' 'Hollywood Ending' "What's Up, Tiger Lily?"
 'Africa Screams' 'Goodbye, Columbus']

Recommendation for user:  71
MovieIds:  ['Best of Enemies' 'Wilde' 'Zeitgeist' 'The World Before Her'
 "Relax... It's Just Sex"
 'Manufacturing Consent: Noam Chomsky and the Media' 'Side by Side'
 'Waiting to Exhale' 'Dracula: Dead and Loving It' 'Cutthroat Island']
Recommendation for user:  316
MovieIds:  ['The Scorpion King: Rise of a Warrior' 'The Swan Princess'
 'Princess Caraboo' 'Star Wars: Episode I - The Phantom Menace'
 'Sinbad and the Eye of the Tiger' 'Dungeons & Dragons'
 'Spacehunter: Adventures in the Forbidden Zone' 'Where Eagles Dare'
 'The Magic Flute' 'The Princess Diaries 2: Royal Engagement']
Recommendation for user:  414
MovieIds:  ['Swimming with Sharks' 'Wag the Dog' 'Be Cool' "Lorna's Silence"
 "Jennifer's Body" 'The Third Man' 'Men with Guns' 'The Player'
 'Trouble in Paradise' 'Dancing in September']
Recommendation for user:  206
MovieIds:  ['Jumanji' 'Grumpier Old Men' 'Waiting to Exhale'
 'Father of the Bride Part II' 'Sabrina' 'Sudden Death' 'GoldenEye'
 'T

Recommendation for user:  366
MovieIds:  ['Mighty Morphin Power Rangers: The Movie' 'Foxfire'
 'Turbo: A Power Rangers Movie' 'The Rapture'
 'The Serpent and the Rainbow' 'The Devil Wears Prada' 'The Namesake'
 'Divergent' 'Fantastic Four' 'Phantasm']
Recommendation for user:  397
MovieIds:  ['Jumanji' 'Grumpier Old Men' 'Waiting to Exhale'
 'Father of the Bride Part II' 'Sabrina' 'Sudden Death'
 'Dracula: Dead and Loving It' 'Cutthroat Island' 'Four Rooms'
 'Ace Ventura: When Nature Calls']
Recommendation for user:  542
MovieIds:  ['Malice' 'Stepmom' 'Toy Story 2' 'Bound for Glory' 'Maid in Manhattan'
 'For Your Consideration' 'Rudolph, the Red-Nosed Reindeer'
 'Rebel Without a Cause' 'Condorman' 'Indecent Proposal']
Recommendation for user:  6
MovieIds:  ['Rescue Dawn' 'The Jacket' 'The Fifth Element' 'Heat' 'The War'
 'The Muppet Christmas Carol' "We're Back! A Dinosaur's Story"
 'Band of the Hand' "Jacob's Ladder" 'Biloxi Blues']
Recommendation for user:  526
MovieIds:  ['Interstel

# Hybrid Approach B (Content-Based and Model-Based Collaborative Filter)

In [87]:
df = pd.DataFrame()
df = pd.merge(model_cf_df, recom_content_pred, how= 'inner', left_on=['userId','movieId'], right_on = ['userId','movieId'])

In [88]:
df_test = pd.DataFrame()
df_test['predictions'] = df['predictions']
df_test['prediction'] = df['prediction']
x_test = df_test.values

In [89]:
#loading model pickle file
with open('hybrid_b.pickle', 'rb') as file:
    pickle_model = pickle.load(file)
    
Ypredict = pickle_model.predict(x_test)

C:\Users\user\anaconda3New\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator LinearRegression from version 0.23.1 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [90]:
df['hybrid_pred'] =  Ypredict 

In [91]:
df.head()

,userId,movieId,predictions,title_x,prediction,title_y,hybrid_pred
0,1,1,2.99,Toy Story,3.538545,Toy Story,3.534684
1,1,2,2.70,Jumanji,3.538545,Jumanji,3.534696
2,1,3,3.06,Grumpier Old Men,3.538545,Grumpier Old Men,3.534681
3,1,4,1.06,Waiting to Exhale,3.538545,Waiting to Exhale,3.534763
4,1,5,2.14,Father of the Bride Part II,3.538545,Father of the Bride Part II,3.534719


### Printing the recommendations for Hybrid Approach B

In [92]:
df= pd.merge(df, movie_title, how='left', left_on=['movieId'], right_on = ['movieId'])

In [93]:
sorted_pred = df.sort_values(by =['userId', 'hybrid_pred'], ascending= False)

In [94]:
users = df.userId.unique()
for i in range(len(users)):
    top10movies = sorted_pred.loc[(sorted_pred['userId'] == users[i])]['title'].values
    top10movies = top10movies[:10]
    print('Recommendation for user: ', users[i])
    print('MovieIds: ',top10movies)

Recommendation for user:  1
MovieIds:  ['Stealth' 'Super Mario Bros.' 'Halloween III: Season of the Witch'
 'Police Academy 5: Assignment Miami Beach' 'Waiting to Exhale'
 'Mighty Morphin Power Rangers: The Movie' 'Battlefield Earth' 'Jaws 3-D'
 "Joe's Apartment" 'Poison Ivy II: Lily']
Recommendation for user:  7
MovieIds:  ['Showgirls' 'Anaconda' 'Dungeons & Dragons' 'Airheads' 'Disaster Movie'
 'Police Academy 5: Assignment Miami Beach' 'Wing Commander'
 'Battlefield Earth' 'Caddyshack II' 'Bio-Dome']
Recommendation for user:  31
MovieIds:  ['My Super Ex-Girlfriend' 'Jason and the Argonauts' 'Jack & Sarah'
 'Only the Strong' 'People, Places, Things'
 'The Bridges of Madison County' 'The Whistleblower' 'Pollyanna'
 'Blood, Guts, Bullets and Octane' 'Suicide Club']
Recommendation for user:  32
MovieIds:  ['Maverick' 'D3: The Mighty Ducks' 'Spy Kids 3-D: Game Over' 'eXistenZ'
 'Ouija' 'The Brothers Grimm' 'Pixels' 'Any Given Sunday' 'Pathology'
 'Zathura: A Space Adventure']
Recommendat

Recommendation for user:  590
MovieIds:  ['Zathura: A Space Adventure' 'Casino Royale' 'Spy Kids 3-D: Game Over'
 'Police Academy 5: Assignment Miami Beach' 'Blade: Trinity'
 'Lost in Space' 'Anaconda' 'The Cable Guy' 'Airheads' 'Super Mario Bros.']
Recommendation for user:  596
MovieIds:  ['Goodbye, Columbus' 'Maid in Manhattan' 'The Messenger' 'Stepmom'
 'Indecent Proposal' "The Ballad of Ramblin' Jack" 'Bound for Glory'
 'Mask' 'Room' 'Rebel Without a Cause']
Recommendation for user:  601
MovieIds:  ['Nutty Professor II: The Klumps' 'Wing Commander' 'Disaster Movie'
 'Battlefield Earth' 'Blade: Trinity' 'Police Academy 6: City Under Siege'
 'Police Academy 3: Back in Training' 'Bio-Dome' 'RoboCop 3'
 'Police Academy 5: Assignment Miami Beach']
Recommendation for user:  607
MovieIds:  ["White Men Can't Jump" 'For Your Consideration' 'Malice'
 'Rebel Without a Cause' 'Match Point' "What's Up, Tiger Lily?"
 'The Messenger' 'Toy Story 3' 'Condorman' 'Indecent Proposal']
Recommendation f

Recommendation for user:  577
MovieIds:  ['The Thin Red Line' 'Condorman' 'Goodbye, Columbus'
 "What's Up, Tiger Lily?" 'Africa Screams' 'For Your Consideration' 'Mask'
 'The Messenger' 'Stanley Kubrick: A Life in Pictures' 'Bringing Up Baby']
Recommendation for user:  582
MovieIds:  ['Watchers' 'The Newton Boys' 'The Muppets Take Manhattan' 'Nine to Five'
 'The Pajama Game' "Charlotte's Web" "Charlotte's Web"
 'Mighty Morphin Power Rangers: The Movie' 'The Pallbearer' 'Stag']
Recommendation for user:  602
MovieIds:  ['Manhunter' 'Dungeons & Dragons' 'Spy Kids 3-D: Game Over'
 'D3: The Mighty Ducks' 'Any Given Sunday' 'Panic Room' 'Ouija'
 'Zathura: A Space Adventure' 'Pixels' 'Gamer']
Recommendation for user:  603
MovieIds:  ['The Thin Red Line' 'Goodbye, Columbus' 'A Midnight Clear'
 "Max Keeble's Big Move" 'Rudolph, the Red-Nosed Reindeer'
 'Indecent Proposal' 'Condorman' 'Toy Story 2' 'Match Point'
 "What's Up, Tiger Lily?"]
Recommendation for user:  605
MovieIds:  ['Moonstruck' 'T

Recommendation for user:  195
MovieIds:  ['Falling Down' 'Night Moves' 'Anaconda' 'Manhunter' 'Benny & Joon'
 'Perfect Stranger' 'Beautiful Thing' 'The Town' 'Legend'
 'Mulholland Falls']
Recommendation for user:  248
MovieIds:  ['Hackers' 'Kafka' 'Event Horizon' 'The Ugly' 'Birdy' "Big Momma's House"
 'Instinct' 'Quarantine' '28 Weeks Later' 'Æon Flux']
Recommendation for user:  276
MovieIds:  ['Wing Commander' 'Police Academy 6: City Under Siege'
 'Police Academy 5: Assignment Miami Beach' 'Anaconda' 'Battlefield Earth'
 'Disaster Movie' 'Showgirls' 'Date Movie' 'Super Mario Bros.'
 'Police Academy 3: Back in Training']
Recommendation for user:  299
MovieIds:  ['Leaving Las Vegas' 'Paradise Now' 'Funny Bones'
 'The Killing of a Chinese Bookie' "Ocean's Eleven" 'The Hangover Part II'
 'Paul Blart: Mall Cop 2' 'Showgirls' 'Last Vegas' 'Diamonds Are Forever']
Recommendation for user:  346
MovieIds:  ['Spirited Away' 'BASEketball' 'D3: The Mighty Ducks'
 'Zathura: A Space Adventure' 'The

Recommendation for user:  391
MovieIds:  ['Roommates' 'Bana Masal Anlatma' 'Descongelate!'
 'Robin Williams: An Evening with Robin Williams' 'The Last Detail'
 'Celebrity' 'Think Like a Man' 'Little Black Book'
 'House on Haunted Hill' 'Dumb and Dumber']
Recommendation for user:  430
MovieIds:  ['Goodbye, Columbus' 'Stanley Kubrick: A Life in Pictures'
 'The Thin Red Line' "White Men Can't Jump" 'Maid in Manhattan'
 'Rudolph, the Red-Nosed Reindeer' 'Match Point' "Max Keeble's Big Move"
 'Bound for Glory' 'The Messenger']
Recommendation for user:  441
MovieIds:  ['Poetic Justice' "Captain Corelli's Mandolin" 'Insurgent'
 'Hells Angels on Wheels' 'Caramel' 'The Longest Week' 'Woo'
 'The Last of England' 'Ali: Fear Eats the Soul' 'Houseboat']
Recommendation for user:  460
MovieIds:  ['Dirty Work' "What's Up, Tiger Lily?" 'For Your Consideration'
 "The Ballad of Ramblin' Jack" 'The Funhouse' 'Rebel Without a Cause'
 'Bringing Up Baby' 'Let It Ride' 'Africa Screams' 'Match Point']
Recommen

...

Recommendation for user:  63
MovieIds:  ['Radio Days' 'For Your Consideration' 'Africa Screams'
 'Rudolph, the Red-Nosed Reindeer' 'Condorman' "White Men Can't Jump"
 'Hollywood Ending' "What's Up, Tiger Lily?" 'The Messenger'
 'The Funhouse']
Recommendation for user:  142
MovieIds:  ['Condorman' "What's Up, Tiger Lily?" 'Africa Screams'
 "White Men Can't Jump" 'Rudolph, the Red-Nosed Reindeer' 'Match Point'
 'The Messenger' 'Radio Days' 'Rebel Without a Cause' 'Toy Story 3']
Recommendation for user:  146
MovieIds:  ['Indecent Proposal' 'Match Point' "White Men Can't Jump" 'Africa Screams'
 'For Your Consideration' 'Hollywood Ending' 'The Funhouse'
 'The Messenger' 'Malice' 'Bound for Glory']
Recommendation for user:  273
MovieIds:  ['Mask' 'Indecent Proposal' 'Condorman' 'Maid in Manhattan'
 "What's Up, Tiger Lily?" 'Africa Screams' "White Men Can't Jump"
 'Rudolph, the Red-Nosed Reindeer' 'For Your Consideration'
 'Hollywood Ending']
Recommendation for user:  286
MovieIds:  ['See Spo

Recommendation for user:  644
MovieIds:  ['The Postman Always Rings Twice' 'The Wizard of Oz' 'Edge of Seventeen'
 'Nutty Professor II: The Klumps' 'Police Academy 6: City Under Siege'
 'Mighty Morphin Power Rangers: The Movie' 'Congo' 'Blade: Trinity'
 'Police Academy 3: Back in Training' 'Wing Commander']
Recommendation for user:  12
MovieIds:  ['Deep Red' 'Leaving Las Vegas' 'Coneheads' 'Wild Wild West'
 "National Lampoon's Loaded Weapon 1" 'Nutty Professor II: The Klumps'
 'Jason X' 'Return to the Blue Lagoon'
 'The Best Little Whorehouse in Texas' '2 Fast 2 Furious']
Recommendation for user:  189
MovieIds:  ['Band of the Hand' 'Novocaine' 'Cocaine Cowboys' 'Point Blank'
 'The Indian in the Cupboard' 'Vampire in Brooklyn' 'Down Periscope'
 'Mighty Morphin Power Rangers: The Movie' 'Strange Days' 'I.Q.']
Recommendation for user:  215
MovieIds:  ['The Funhouse' 'Goodbye, Columbus' 'Let It Ride' 'The Thin Red Line'
 'Africa Screams' 'For Your Consideration' 'Blade: Trinity'
 'Wing Com

Recommendation for user:  448
MovieIds:  ['Rebel Without a Cause' 'Battlefield Earth' 'Howard the Duck'
 'Poltergeist' 'Mighty Morphin Power Rangers: The Movie'
 'Halloween III: Season of the Witch' 'Waiting to Exhale'
 "Joe's Apartment" "Big Momma's House" 'Jaws 3-D']
Recommendation for user:  456
MovieIds:  ['Goodbye, Columbus' 'Mask' 'Rebel Without a Cause'
 'The Blair Witch Project' 'The Haunting' 'Super Mario Bros.' 'Joe Dirt'
 "Joe's Apartment" 'Jaws 3-D' 'Disaster Movie']
Recommendation for user:  486
MovieIds:  ['Indecent Proposal' 'Africa Screams' 'Goodbye, Columbus'
 "What's Up, Tiger Lily?" 'Malice' 'Hollywood Ending' 'Radio Days'
 'Rudolph, the Red-Nosed Reindeer' 'Condorman' 'The Funhouse']
Recommendation for user:  512
MovieIds:  ['Get Hard' 'No Escape' 'American Gigolo' 'Need for Speed'
 'The Human Centipede 3 (Final Sequence)' 'You Only Live Once'
 'The Getaway' 'Brubaker' 'The Longest Yard' 'Starred Up']
Recommendation for user:  541
MovieIds:  ['Life With Mikey' 'Spun

Recommendation for user:  600
MovieIds:  ['Killing Zoe' 'Crouching Tiger, Hidden Dragon' 'Penguins of Madagascar'
 'George of the Jungle' 'Beach Blanket Bingo'
 'Sinbad and the Eye of the Tiger' 'Raising Arizona' 'Deadtime Stories'
 "Kiki's Delivery Service" 'Tristan & Isolde']
Recommendation for user:  71
MovieIds:  ['Best of Enemies' 'Wilde' 'Zeitgeist' 'The World Before Her'
 'Side by Side' "Relax... It's Just Sex"
 'Manufacturing Consent: Noam Chomsky and the Media' 'Blade: Trinity'
 'Halloween III: Season of the Witch' 'Battlefield Earth']
Recommendation for user:  316
MovieIds:  ['The Scorpion King: Rise of a Warrior' 'Dungeons & Dragons'
 'Princess Caraboo' 'Where Eagles Dare' 'The Swan Princess'
 'The Princess Diaries 2: Royal Engagement'
 'Sinbad and the Eye of the Tiger'
 'Spacehunter: Adventures in the Forbidden Zone' 'The Magic Flute'
 'Mirror Mirror']
Recommendation for user:  414
MovieIds:  ['Be Cool' "Lorna's Silence" 'Wag the Dog' "Jennifer's Body"
 'Dancing in Septembe

Recommendation for user:  610
MovieIds:  ['Carbon Copy' 'Jaws: The Revenge' 'Tekkonkinkreet' 'Made in America'
 'Scream 2' 'Jupiter Ascending' 'The Opposite of Sex' 'Ragtime'
 'Shoot to Kill' 'Black and White']
Recommendation for user:  366
MovieIds:  ['Mighty Morphin Power Rangers: The Movie' 'Divergent'
 'Turbo: A Power Rangers Movie' 'The Butterfly Effect' 'The Rapture'
 'Fantastic Four' 'The Namesake' 'Phantasm' 'The Devil Wears Prada'
 'The Serpent and the Rainbow']
Recommendation for user:  397
MovieIds:  ['Anaconda' 'Wing Commander' 'Police Academy 5: Assignment Miami Beach'
 'Battlefield Earth' 'Disaster Movie' 'Blade: Trinity' 'Jaws 3-D'
 'Barb Wire' 'Cutthroat Island' 'Ace Ventura: When Nature Calls']
Recommendation for user:  542
MovieIds:  ['Maid in Manhattan' 'Rudolph, the Red-Nosed Reindeer' 'Stepmom'
 'Toy Story 2' 'Bound for Glory' 'For Your Consideration' 'Malice'
 'Indecent Proposal' 'Match Point' 'Rebel Without a Cause']
Recommendation for user:  6
MovieIds:  ['Rescu

# Hybrid Approach C (Content-Based and User-Based Collaborative Filter)

In [95]:
df = pd.DataFrame()
df = pd.merge(user_cf_df, recom_content_pred, how= 'inner', left_on=['userId','movieId'], right_on = ['userId','movieId'])

In [96]:
df_test = pd.DataFrame()
df_test['predictions'] = df['predictions']
df_test['prediction'] = df['prediction']
x_test = df_test.values

In [97]:
#loading model pickle file
with open('hybrid_c.pickle', 'rb') as file:
    pickle_model = pickle.load(file)
    
Ypredict = pickle_model.predict(x_test)

C:\Users\user\anaconda3New\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator LinearRegression from version 0.23.1 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [98]:
df['hybrid_pred'] =  Ypredict 

In [99]:
df.head()

,userId,movieId,predictions,title_x,prediction,title_y,hybrid_pred
0,1,1,2.69,Toy Story,3.538545,Toy Story,3.580863
1,1,2,2.20,Jumanji,3.538545,Jumanji,3.598008
2,1,3,1.86,Grumpier Old Men,3.538545,Grumpier Old Men,3.609905
3,1,4,1.32,Waiting to Exhale,3.538545,Waiting to Exhale,3.628799
4,1,5,2.08,Father of the Bride Part II,3.538545,Father of the Bride Part II,3.602207


### Printing the recommendations for Hybrid Approach C

In [100]:
df= pd.merge(df, movie_title, how='left', left_on=['movieId'], right_on = ['movieId'])

In [101]:
sorted_pred = df.sort_values(by =['userId', 'hybrid_pred'], ascending= False)

In [102]:
users = df.userId.unique()
for i in range(len(users)):
    top10movies = sorted_pred.loc[(sorted_pred['userId'] == users[i])]['title'].values
    top10movies = top10movies[:10]
    print('Recommendation for user: ', users[i])
    print('MovieIds: ',top10movies)

Recommendation for user:  1
MovieIds:  ['Stealth' 'Across the Sea of Time' 'Bio-Dome'
 'Lawnmower Man 2: Beyond Cyberspace' 'Dunston Checks In'
 "A Midwinter's Tale" 'Frankie Starlight' 'Man of the Year'
 'Free Willy 2 - The Adventure Home'
 'Mighty Morphin Power Rangers: The Movie']
Recommendation for user:  7
MovieIds:  ["A Midwinter's Tale" 'Man of the Year' 'In the Army Now' 'Savage Nights'
 'Andre' 'The Superwife' 'Yes, Madam' 'Dangerous Ground'
 'Amityville: Dollhouse' 'Amityville: A New Generation']
Recommendation for user:  31
MovieIds:  ['My Super Ex-Girlfriend' 'Jason and the Argonauts' 'Jack & Sarah'
 'People, Places, Things' 'Only the Strong'
 'Blood, Guts, Bullets and Octane' 'Suicide Club'
 'The Bridges of Madison County' 'The Whistleblower' 'Sex Ed']
Recommendation for user:  32
MovieIds:  ['Maverick' 'Spy Kids 3-D: Game Over' 'Ouija' 'D3: The Mighty Ducks'
 'Pathology' 'Pixels' 'The Brothers Grimm' 'Gamer' 'eXistenZ'
 'Any Given Sunday']
Recommendation for user:  36
Mov

Recommendation for user:  601
MovieIds:  ["A Midwinter's Tale" 'The Superwife' 'Butterfly Kiss' 'Dangerous Ground'
 'Turbo: A Power Rangers Movie' 'Stag' "Barney's Great Adventure"
 'Psycho III' 'Carnosaur 3: Primal Species' 'Last Exit to Brooklyn']
Recommendation for user:  607
MovieIds:  ['Malice' 'For Your Consideration' "White Men Can't Jump"
 'Rebel Without a Cause' 'Match Point' "What's Up, Tiger Lily?"
 'The Messenger' 'Condorman' 'Toy Story 3' 'Indecent Proposal']
Recommendation for user:  619
MovieIds:  ['Muppet Treasure Island' "A Midwinter's Tale" 'Man of the Year'
 'In the Army Now' 'Savage Nights' 'Andre' 'The Superwife' 'Yes, Madam'
 'Dangerous Ground' 'Amityville: Dollhouse']
Recommendation for user:  639
MovieIds:  ['Kansas' 'Suspect' "Don't Tell Mom the Babysitter's Dead" 'Shaft'
 'Senior Trip' 'Sleuth' 'Georgia' 'Mildred Pierce' '3:10 to Yuma'
 'Padre Padrone']
Recommendation for user:  641
MovieIds:  ['Mask' 'Indecent Proposal' 'Dirty Work' "What's Up, Tiger Lily?" '

Recommendation for user:  23
MovieIds:  ['Stanley Kubrick: A Life in Pictures' 'Stepmom'
 "The Ballad of Ramblin' Jack" 'Tai-Chi Master' 'Maid in Manhattan' 'Room'
 'Malice' "White Men Can't Jump" 'Smiles of a Summer Night' 'Let It Ride']
Recommendation for user:  30
MovieIds:  ['A Midnight Clear' 'Match Point' 'Tai-Chi Master' 'Room'
 'For Your Consideration' 'Bound for Glory' 'Mask' 'Toy Story 3'
 'Hollywood Ending' 'Dirty Work']
Recommendation for user:  70
MovieIds:  ['Indecent Proposal' "What's Up, Tiger Lily?" 'Africa Screams' 'Malice'
 'Hollywood Ending' 'For Your Consideration' "White Men Can't Jump"
 'Match Point' 'Bound for Glory' 'Condorman']
Recommendation for user:  90
MovieIds:  ['Indecent Proposal' "What's Up, Tiger Lily?" 'Africa Screams' 'Malice'
 'Hollywood Ending' 'For Your Consideration' "White Men Can't Jump"
 'Match Point' 'Toy Story 2' 'Bound for Glory']
Recommendation for user:  105
MovieIds:  ['Death Wish 2' 'The Crush' 'Mulholland Falls' 'Escape from L.A.'
 'T

Recommendation for user:  353
MovieIds:  ['Rebel Without a Cause' 'Malice' 'Across the Sea of Time'
 'Dunston Checks In' "A Midwinter's Tale" 'Man of the Year'
 'Mighty Morphin Power Rangers: The Movie' 'Destiny Turns on the Radio'
 'Dangerous Game' 'In the Army Now']
Recommendation for user:  358
MovieIds:  ['The Thin Red Line' 'Bound for Glory' 'Smiles of a Summer Night'
 'Rebel Without a Cause' 'The Funhouse' 'Indecent Proposal'
 'The Messenger' 'Condorman' 'Toy Story 3' 'For Your Consideration']
Recommendation for user:  390
MovieIds:  ['Let It Ride' 'Indecent Proposal' "What's Up, Tiger Lily?"
 'Africa Screams' 'Malice' 'Hollywood Ending' 'For Your Consideration'
 'Match Point' 'Condorman' 'The Messenger']
Recommendation for user:  423
MovieIds:  ['Donnie Brasco' 'Things Change' 'İtirazım Var'
 'Ghost Dog: The Way of the Samurai' 'Think Like a Man Too' 'Tracers'
 'Bad Lieutenant' 'Paul Blart: Mall Cop 2' 'Bugsy' 'The Electric Horseman']
Recommendation for user:  463
MovieIds:  ['R

Recommendation for user:  497
MovieIds:  ['Above the Rim' 'He Got Game' 'American Teen' 'All the Right Moves'
 'Grown Ups' 'The Woodsman' 'Step Up' 'Dreamscape' 'Win Win'
 'Love & Basketball']
Recommendation for user:  505
MovieIds:  ['Syrup' 'Houseboat' 'Tap' "Captain Corelli's Mandolin"
 "A Midwinter's Tale" 'Man of the Year' 'In the Army Now' 'Savage Nights'
 'Ciao, Professore!' 'The Superwife']
Recommendation for user:  510
MovieIds:  ['A Midnight Clear' 'Mask' 'Bringing Up Baby' 'Stepmom'
 "A Midwinter's Tale" 'Man of the Year' 'In the Army Now' 'Savage Nights'
 'Ciao, Professore!' 'The Superwife']
Recommendation for user:  521
MovieIds:  ['High and Low' 'Four Lions' 'Still Breathing'
 'The Dangerous Lives of Altar Boys' 'Daylight' 'Like Water for Chocolate'
 'The Wrong Man' 'Throne of Blood' 'Nightcrawler' 'Hotel Chevalier']
Recommendation for user:  539
MovieIds:  ['People, Places, Things' 'Bride & Prejudice' 'Blame It on Rio'
 'Lovely & Amazing' 'Pride and Prejudice' 'Arthur' '

...

Recommendation for user:  237
MovieIds:  ["A Midwinter's Tale" 'Man of the Year' 'In the Army Now' 'Savage Nights'
 'Ciao, Professore!' 'The Superwife' "Joe's Apartment" 'Yes, Madam'
 'Dangerous Ground' 'T-Men']
Recommendation for user:  244
MovieIds:  ['Saints and Soldiers' 'Tea with Mussolini' 'Mrs. Miniver'
 'The Last of the Mohicans' 'The Lords of Flatbush' 'Separate Lies'
 'Village of the Damned' "A Midwinter's Tale" 'Amityville: Dollhouse'
 "Barney's Great Adventure"]
Recommendation for user:  251
MovieIds:  ['Mirror' 'King Ralph' 'Broken' "The Blood on Satan's Claw" 'Henry V'
 'Separate Lies' 'Village of the Damned' 'The Lords of Flatbush'
 'Straw Dogs' 'Stalingrad']
Recommendation for user:  277
MovieIds:  ['King Ralph' 'Separate Lies' 'Straw Dogs' 'Stalingrad' 'Ambush'
 'The New World' 'The Last of the Mohicans' 'Corpse Bride'
 'Certified Copy' 'The Lords of Flatbush']
Recommendation for user:  293
MovieIds:  ['From Beyond' 'Howard the Duck' 'Memories' 'The Ice Pirates'
 'Harr

Recommendation for user:  53
MovieIds:  ["Jacob's Ladder" 'The Butterfly Effect 3: Revelations' 'Band of the Hand'
 'Chinese Take-Out' 'The War' 'Race to Witch Mountain'
 'Murmur of the Heart' 'The Muppet Christmas Carol' 'Heat' 'Biloxi Blues']
Recommendation for user:  250
MovieIds:  ['Race to Witch Mountain' "We're Back! A Dinosaur's Story"
 'Band of the Hand' 'Chinese Take-Out' 'The War' 'Murmur of the Heart'
 'The Fifth Element' 'The Muppet Christmas Carol' 'Heat' 'Pek Yakında']
Recommendation for user:  531
MovieIds:  ["A Midwinter's Tale" 'Man of the Year' 'In the Army Now' 'Savage Nights'
 'Ciao, Professore!' 'The Superwife' 'Kim' "Joe's Apartment" 'Yes, Madam'
 'Dangerous Ground']
Recommendation for user:  174
MovieIds:  ['The Indian in the Cupboard' 'DragonHeart' 'When a Man Loves a Woman'
 'Dance with Me' 'Jack the Giant Slayer' 'Pollyanna' 'Planet 51'
 'Swiss Family Robinson' 'American Pop' 'Return to Never Land']
Recommendation for user:  652
MovieIds:  ['The Cell' 'Mumford

Recommendation for user:  140
MovieIds:  ['Romance' 'The Out of Towners' 'Funny People' 'Miss Congeniality'
 'Killer' 'Hollywoodland' 'Niagara' 'George of the Jungle'
 'Two Weeks Notice' 'Real Women Have Curves']
Recommendation for user:  166
MovieIds:  ['Crimes of the Heart' 'The Advocate' 'Fraternity Vacation'
 'Forever Young' 'The Skin I Live In' 'Nine to Five'
 "City Slickers II: The Legend of Curly's Gold" 'Lassie' 'Mr. Deeds'
 'The Black Cauldron']
Recommendation for user:  234
MovieIds:  ['Dungeons & Dragons' 'Short Circuit 2' "Bill & Ted's Bogus Journey"
 'Making Mr. Right' 'The Scorpion King: Rise of a Warrior'
 'Sinbad and the Eye of the Tiger' 'Princess Caraboo' 'Mirror Mirror'
 'The Princess Diaries 2: Royal Engagement'
 'Spacehunter: Adventures in the Forbidden Zone']
Recommendation for user:  238
MovieIds:  ['Frankie Starlight' "Harry, He's Here To Help" 'Miracle Mile'
 'In America' "A Midwinter's Tale" 'In the Army Now' 'The Superwife'
 'Dangerous Ground' 'Amityville: Do

Recommendation for user:  117
MovieIds:  ['The Pirate Movie' 'Hereafter' 'Bed of Roses' 'Before Midnight'
 'Tea with Mussolini' 'Crash' 'Amityville: Dollhouse' 'Stag'
 "Barney's Great Adventure" 'Psycho III']
Recommendation for user:  169
MovieIds:  ['Goodbye, Columbus' 'The Thin Red Line' "Max Keeble's Big Move" 'Room'
 'Radio Days' 'Indecent Proposal' "What's Up, Tiger Lily?"
 'Africa Screams' 'Malice' 'Hollywood Ending']
Recommendation for user:  331
MovieIds:  ['Turbo: A Power Rangers Movie' 'Mighty Morphin Power Rangers: The Movie'
 'Fantastic Four' 'Divergent' 'The Rapture' 'The Namesake' 'Foxfire'
 'The Serpent and the Rainbow' 'The Devil Wears Prada'
 'The Butterfly Effect']
Recommendation for user:  347
MovieIds:  ['Amityville: Dollhouse' "Barney's Great Adventure" 'Psycho III'
 'Worth Winning' 'The Principal' 'One from the Heart'
 "Malibu's Most Wanted" 'An Angel at My Table' 'To Be and to Have'
 'The Front Page']
Recommendation for user:  376
MovieIds:  ['Raw Deal' 'Need for

Recommendation for user:  29
MovieIds:  ['The Principal' 'Love & Other Drugs' 'Walking Tall' 'Tequila Sunrise'
 'Journey to the Center of the Earth' 'Traffic'
 'The Prime of Miss Jean Brodie' "Carlito's Way" 'Brubaker' "Bush's Brain"]
Recommendation for user:  179
MovieIds:  ['Maid in Manhattan' 'Indecent Proposal' "What's Up, Tiger Lily?"
 'Africa Screams' 'Malice' 'Hollywood Ending' 'For Your Consideration'
 "White Men Can't Jump" 'Match Point' 'Bound for Glory']
Recommendation for user:  374
MovieIds:  ['Man of the Year' 'In the Army Now' 'Savage Nights' 'Andre'
 'The Superwife' "Joe's Apartment" 'Yes, Madam'
 'Children of the Corn IV: The Gathering' 'T-Men' 'Amityville: Dollhouse']
Recommendation for user:  65
MovieIds:  ['The Scorpion King: Rise of a Warrior' 'The Magic Flute'
 'The Swan Princess' 'Princess Caraboo' 'Mirror Mirror'
 "Bill & Ted's Bogus Journey"
 'Spacehunter: Adventures in the Forbidden Zone'
 'Kingsglaive: Final Fantasy XV' 'Shanghai Noon'
 'The Princess Diaries 